# TODO


constractive destilattion based on softmax predicts of cosine similarity and a distanece
$=\exp{[ s(p_T,p_S])}/\sum_i\exp{[s(p^{(i)}_T,p^{(i)}_S)]}$


pipelines
- first investigation
  - diferent modelsizes
- optimization
  - sparse, quantize, distillation, (sparse + quantize), (sparse + quantize+ lora), (distillation + sparse + quantize+ lora)

- data aumentation based in the ranked
- apply finds to all datasets
  - select datasets
- write saves in images to drive





In [ ]:
# !pip install megatron-core


# https://github.com/NVIDIA/Megatron-LM/commit/555036e88fa7129e7caaf67b0c1ec838703c35dd#diff-3a709bfafbc79eb07662cc059fadc6b8cca7aa6083f16831fd029865636bf298
# https://github.com/huggingface/peft/blob/bf54136a79cc85b0e4c3915b4e1eb158f43c4b73/src/peft/tuners/lora/tp_layer.py#L26

# from megatron.core.tensor_parallel.layers import ColumnParallelLinear, RowParallelLinear

# core.transformer.transformer_config.

In [ ]:
# cuda torch.copile() aceleration


# !export LC_ALL="en_US.UTF-8"
# !export LD_LIBRARY_PATH="/usr/lib64-nvidia"
# !export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
# !ldconfig /usr/lib64-nvidia

# Init

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
path = "/content/gdrive/MyDrive/Projetos/TCC/slr_data_tcc.zip"

In [ ]:
!wget -O data.zip https://www.dropbox.com/sh/or0eyfo8znyu2kp/AABxXJVII48U0vY8TT3Bbp6Ea?dl=0
!unzip "./gdrive/MyDrive/Projetos/TCC/slr_data_tcc.zip"

# !unzip OriData/DATA_SLR.zip

In [ ]:
# !pip install -U adapter-transformers peft
# !pip install -q peft
!pip install -q einops einop
!pip install -q bitsandbytes accelerate datasets
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
!pip install -q git+https://github.com/huggingface/accelerate.git@main

# !pip install -U adapter-transformers

# Imports


In [ ]:
import argparse
import json
import logging
import math
import os
import random
from pathlib import Path
from tqdm import tqdm

# import datasets
# from datasets import load_dataset, DatasetDict

# import evaluate
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import transformers

# from google.colab import userdata
# userdata.get('HF_TOKEN')

##  Utils functions

In [ ]:
import json

def read_json(file_path):
    """
    Reads JSON data from a file.

    Args:
    - file_path (str): The path to the JSON file.

    Returns:
    - data (dict): The JSON data read from the file.
    """
    try:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
        return data
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in file '{file_path}'.")
        return None


def create_subset(original_dict, keys_to_keep):
    """
    Create a subset dictionary containing only the specified keys from the original dictionary.

    Args:
    - original_dict (dict): The original dictionary.
    - keys_to_keep (list): A list of keys to keep in the subset dictionary.

    Returns:
    - subset_dict (dict): The subset dictionary containing only the specified keys.
    """
    subset_dict = {}
    for key in keys_to_keep:
        if key in original_dict:
            subset_dict[key] = original_dict[key]
    return subset_dict


import os
def data_import(file_path, n=8, samples_n=200, samples_p=1):
    print("####"*5)
    print()
    print(os.path.basename(file_path))
    df = preprocess_csv_file(file_path)


    tk = lambda x: len(tokenizer_t_ref.encode(x))
    tk_len = df.text.map(tk)
    tk_len.hist()


    df_rest = df[tk_len > 20]
    df_train = df_rest.groupby('label').sample(n)

    print("data count:")
    print(df['label'].value_counts())

    print(df_train.iloc[0])

    df_test = df[~df.index.isin(df_train.index)]

    if len(df_test[df_test['label'] == 1]) < samples_p:
        df_test_one = df_test[df_test['label'] == 1].copy()
    else:
        df_test_one = df_test[df_test['label'] == 1].sample(n=samples_p, replace=False)


    if len(df_test[df_test['label'] == 0]) < samples_n:
        df_test_zero = df_test[df_test['label'] == 0].copy()
    else:
        df_test_zero = df_test[df_test['label'] == 0].sample(n=samples_n, replace=False)

    exs_test = pd.concat([df_test_one,df_test_zero])

    print("Eval count:")
    print(exs_test['label'].value_counts())
    return df_train, exs_test


def loaders(df_train, exs_test, config):
    sim_data = SIMDataset(df_train, tokenize_batch_t, config["n_pairs"],
                          )
    train_dataloader = DataLoader(sim_data,shuffle=True,
                                  batch_size=config["batch_size"])

    test_dataloader_eval = DataLoader(shuffle=False, batch_size=config["batch_size_eval"],
                dataset=SIMDataset(
                    exs_test, tokenize_batch_t, 0,
                          )
                )

    train_dataloader_eval = DataLoader(shuffle=False, batch_size=config["batch_size"],
                dataset=SIMDataset(
                    df_train, tokenize_batch_t, 0,
                    # emb= emb, device="cuda", pttn="* d a"
                          )
                )

    return train_dataloader, test_dataloader_eval,  train_dataloader_eval


In [ ]:
import os

def create_repo_if_not_exist(repo_path):
    directory = os.path.dirname(repo_path)
    os.makedirs(directory, exist_ok=True)
    print("created!",repo_path)



In [ ]:
import torch
import os

def count_non_zero_params(model):
    total_params = 0
    non_zero_params = 0

    for param in model.parameters():
        to = param.numel()
        non = torch.count_nonzero(param).item()
        non = 0 if non is None else non
        # print(non)
        # print(to)
        # print(non/to)

        total_params += to  # Count total number of parameters
        non_zero_params += non  # Count non-zero elements
        # print(non)
        # print(non/to)*100


    percentage_non_zero = (non_zero_params / total_params) * 100

    return percentage_non_zero

def export_model(model, file_path):
    """
    Save a PyTorch model to a file.

    Parameters:
        model (state_dict): The PyTorch state_dict of the model to be saved.
        file_path (str): Path to the file where the model will be saved.

    Returns:
        bool: True if the model is successfully saved, False otherwise.
    """
    try:
        # Create directory if it doesn't exist
        directory = os.path.dirname(file_path)
        os.makedirs(directory, exist_ok=True)

        # Save the model
        torch.save(model, file_path)
        print(f"Exported Model: {file_path}")
    except Exception as e:
        print(f"Error saving model: {e}")
        return False


import json
import os

def numpy_to_list(d):
    """
    Recursively convert NumPy arrays to lists in a dictionary.

    Parameters:
        d (dict): Dictionary to be processed.

    Returns:
        dict: Dictionary with NumPy arrays converted to lists.
    """
    for key, value in d.items():
        if isinstance(value, dict):
            d[key] = numpy_to_list(value)
        elif isinstance(value, np.ndarray):
            d[key] = value.tolist()
    return d

def export_json(data, file_path):
    """
    Export data to a JSON file.

    Parameters:
        data (dict): Data to be exported to JSON.
        file_path (str): Path to the JSON file.

    Returns:
        bool: True if the export is successful, False otherwise.
    """
    data = numpy_to_list(data)
    try:
        # Create directory if it doesn't exist
        directory = os.path.dirname(file_path)
        os.makedirs(directory, exist_ok=True)

        # Export data to JSON file
        with open(file_path, 'w') as json_file:
            json.dump(data, json_file, indent=2)
        print(f"Exported to JSON: {file_path}")
    except Exception as e:
        print(f"Error exporting to JSON: {e}")
        # return False



In [ ]:
import pandas as pd

def preprocess_csv_file(file_path):
    """Preprocess CSV file."""
    # Carregar o arquivo CSV em um DataFrame do pandas
    df = pd.read_csv(file_path)

    # # Exibir as primeiras linhas do DataFrame
    # df.head(2)

    df = df.rename(columns={
        "label_included": "label",
        "labels": "label"
        })

    col = "title"
    df.loc[df[col].isna(), col] = ""
    col = "abstract"
    df.loc[df[col].isna(), col] = ""

    df['text'] = df['title'] + "; " + df['abstract']


    return df


In [ ]:
from torch.utils.data import DataLoader
from einops import pack, unpack

n = 50
ex = 2
input_data = torch.randn(n,ex, 5)
output_data = torch.randn(n, 1)
batch = 20


def batch_load(input, output = None, batch=32):
  input_data = input.get('data')
  output_data = output.get('data')

  input_cfg = input.get('config')
  output_cfg = output.get('config')


  full = input_data.shape[0]//batch
  batch_as = [[batch]]*full
  rest =  input_data.shape[0]%batch

  if rest > 0:
    batch_as += [[rest]]

  # input_data.shape
  in_a = unpack(input_data, batch_as, input_cfg)
  out_a = unpack(output_data, batch_as, output_cfg)

  return zip(in_a, out_a)







a = batch_load(
    input = {
        "data": input_data,
        "config": "* ex x"
    },
    output = {
        "data": output_data,
        "config": "* y"
    },
    batch = batch
)




b = iter(a)
x = next(b)

# Mods

todo:
fazer a interação por loop para compartilhar os pesos junto ao copiar a arquitetra, ao inves de copiar o modelo todo de uma vez

## 📣 ModUtils

### Main class

In [ ]:
import torch.nn as nn
import torch
import copy
from collections import OrderedDict


import torch
import torch.nn as nn

import re


####### ajustar call para conseguir utilizar do modular depois do map


class ModularUtils():
    # Modular = Lora
    # type_Mod = nn.Linear
    # config=None
    # class_mod = ModularUtils_modules

    def __call__(self, module):

      ModularUtils(
                 module = module,
                 Modular=self.Modular,
                 type_Mod=self.type_Mod,
                 config=self.config,
                 tag=self.tag)


      print(f'[{self.tag}.util] Done!')


    def __init__(self,
                 module = None,
                 Modular=None,
                 type_Mod=nn.Linear,
                 config=None,
                 tag=''
                 ):
      """
      Initialize ModularUtils.

      Args:
          Modular (callable): The custom module to replace layers.
          type_Mod (type): The type of module to replace.
          config (dict): Configuration for the custom module.
      """

      self.Modular = Modular
      self.type_Mod = type_Mod
      self.config=config
      self.tag=tag
      self.module=module

      if module is not None:
        module.util = self


    # def sub_all_linear(self, module,sub_linear=Linear4bit, **kargs):
    #     module = self.module if (module is None) else module
    #     assert (module is not None), "pass a module"

    #     for name, sub_module in fp16_model.named_modules():
    #         if isinstance(sub_module, nn.Linear):
    #             lshape = module.weight.shape
    #             setattr(module, name, sub_linear(*lshape, **kargs))


    def map(self,
            name_module=None,
            module=None,
            share_weights = True,
            **kargs):
        """
        Recursively replaces layers in a module with a custom module.

        Args:
            module (nn.Module): The input module to be modified.
            name_module (list, optional): List of names to filter modules.
            share_weights (bool, optional): If True, apply the new module on parallel model with shared weights between the original and the adapted model modules.
            load_by_layer (bool, optional): If True, loads state_dict layer by layer (used with share_weights).
            **kwargs: Additional keyword arguments.
                - Modular (callable): The custom module to replace layers.
                - type_Mod (type): The type of module to replace.
                - config (dict): Configuration for the custom module.

        Returns:
            nn.Module: The modified module.
        """
        # config:
        self.Modular = kargs.get("Modular", self.Modular)
        self.type_Mod = kargs.get("type_Mod", self.type_Mod)
        config = kargs.get("config", self.config)
        config = {} if config is None else config

        module = self.module if (module is None) else module
        assert (module is not None), "pass a module"

        if share_weights:
          names, sub_modules =  self.Modular_rec(module, name_module, config)
          return module, (names, sub_modules)
        else:
          module_adapt=copy.deepcopy(module)
          names, sub_modules = self.Modular_rec(module_adapt , name_module, config)
          return module_adapt, (names, sub_modules)


    def Modular_rec(self, model, patterns, config):
      def replace_module_by_path(model, module_path, new_module):

        # Split the module path string
        parts = module_path.split('.')

        # print(parts)
        # Access the module using getattr() with the split parts
        current_module = model
        for part in parts[:-1]:
            current_module = getattr(current_module, part)

        # Replace the existing module with the new module
        setattr(current_module, parts[-1], new_module)


      modules = dict(model.named_modules())
      matched_modules = []
      matched_pttn = []

      def main(model, name):
          Modular_module = self.Modular(module, **config)

          matched_pttn.append(name), matched_modules.append(Modular_module)

          replace_module_by_path(model,name , Modular_module)


      if patterns is None:
          for name, module in modules.items():
            if isinstance(module, self.type_Mod):
              main(model, name)

      elif isinstance(patterns, list):
        for pattern in patterns:
            for name, module in modules.items():
              if re.search(pattern, name) and isinstance(module, self.type_Mod):
                if not name in matched_pttn:
                  main(model, name)
      elif isinstance(patterns, str):
        pattern = patterns
        for name, module in modules.items():
          if re.search(pattern, name) and isinstance(module, self.type_Mod):
            if not name in matched_pttn:
              main(model, name)


      return matched_pttn, matched_modules


    # def Modular_rec(self, module, name_module=None, name=None, parent_name=None, full_names=None):
    #     if full_names is None:
    #       full_names = []

    #     if isinstance(module, self.type_Mod):
    #         if name_module is None or (name and name_module is not None and name in name_module):
    #             Modular_module = self.Modular(module, self.config)
    #             full_names.append(parent_name)
    #             return Modular_module, full_names

    #     for name, sub_module in module.named_children():

    #         # Construct the full name of the submodule
    #         full_name = f"{parent_name}.{name}" if parent_name else name

    #         if isinstance(sub_module, nn.ModuleDict):
    #             # Handle nn.ModuleDict by iterating through values
    #             for key_sub, value_sub in sub_module.items():
    #                 sub_module[key_sub], full_names = self.Modular_rec(value_sub, name_module, name, full_name, full_names)
    #         elif isinstance(sub_module, nn.ModuleList):
    #             # Handle nn.ModuleList by iterating through elements
    #             for i, elem_sub in enumerate(sub_module):
    #                 sub_module[i], full_names = self.Modular_rec(elem_sub, name_module, name, full_name, full_names)
    #         else:
    #             # Handle other cases
    #             sub_module, full_names = self.Modular_rec(sub_module, name_module, name, full_name, full_names)


    #         setattr(module, name, sub_module)

    #     return module, full_names

    @staticmethod
    def load_state_dict_layer_by_layer(state_dict, module):
        """
        Loop over layers to load weights to a module.

        Args:
            module (nn.Module): The input module to load state_dict.
            state_dict (dict): Dict of weights to load layer by layer in modules.
        """

        for name, param in module.named_parameters():
            if name in state_dict:
                param.data.copy_(state_dict[name])
            else:
                print(f"Warning: Parameter {name} not found in the state_dict.")


    def export_mod(self,
                           join=False,
                           module= None,
                           names=[],
                           **kargs):
        module = self.module if (module is None) else module
        assert (module is not None), "pass a module"

        """
        Export the weights of the Modular layers in the module.

        Args:
            module (nn.Module): The input module to export the weights of Modular state_dict.
            join (bool): True if is a OrderedDict or False to a dict of OrderedDicts.
        """

        Modular = kargs.get("Modular", self.Modular)

        def concatenate_keys(input_dict, separator='.'):
            result_dict = OrderedDict()

            for key, value in input_dict.items():
                if isinstance(value, dict):
                    nested_dict = concatenate_keys(value, separator)
                    for nested_key, nested_value in nested_dict.items():
                        result_key = f"{key}{separator}{nested_key}"
                        result_dict[result_key] = nested_value
                else:
                    result_dict[key] = value

            return result_dict


        all_weights = {}
        for name, module in module.named_modules():
            if isinstance(module, Modular):
                if len(names)>0:
                  print(name)
                  if name.split(".")[-1] in names:
                    all_weights[name] = module.state_dict()
                else:
                    all_weights[name] = module.state_dict()

        if join:
          all_weights = concatenate_keys(all_weights)
        return all_weights


    def load_mod(self,
                 all_weights=None,
                 module=None,
                 **kargs):
      """
      Export the weights of the Modular layers in the module.

      Args:
          module (nn.Module): The input module to export the weights of Modular state_dict.
          all_weights (dict): Dict of  OrderedDicts.

      Exemple:
          module

          module.load_state_dict(all_weights) # if all_weights is OrderedDict to apply.
          load_mod_weights(module, all_weights) # if all_weights is a dict of OrderedDict weights.
      """
      module = self.module if (module is None) else module
      assert (module is not None), "pass a module"


      Modular = kargs.get("Modular", self.Modular)
      for name, module in module.named_modules():
          if isinstance(module, Modular):
              module.load_state_dict(all_weights[name])


    # target_path be your own
    def freeze(self, target_paths=None, freeze=True, module=None):
        """
        Freeze or unfreeze layers in a PyTorch module based on the specified paths.

        Args:
            module (nn.Module): The PyTorch module.
            target_paths (list, optional): A list of paths to the target layers in the module. If None, freeze or unfreeze all layers. Default is None.

        Returns:
            None
        """
        module = self.module if (module is None) else module
        assert (module is not None), "pass a module"

        freezed = ()
        if target_paths is None:
            # Freeze or unfreeze all layers in the module
            for param in module.parameters():
                param.requires_grad = not freeze
        else:
            # Freeze or unfreeze the specified layers
            for target_path in target_paths:
                current_module = module
                keys = target_path.split('.')

                for key in keys[:-1]:
                    current_module = getattr(current_module, key)

                target_layer = getattr(current_module, keys[-1])

                for param in target_layer.parameters():
                    param.requires_grad = not freeze

    @staticmethod
    def meta_fun(objects, methods, args):
        for obj, method_name, arg in zip(objects, methods, args):
            method = getattr(obj, method_name)
            if callable(method):
                return method(arg)
            else:
                print(f"Object {obj} does not have method {method_name}")


    def activate(self, layer_names=None, key=None, module=None, config={}, **karg):
        # print(karg)
        module = self.module if (module is None) else module
        assert (module is not None), "pass a module"

        Modular = karg.get('Modular', self.Modular)

        returns = ()
        for name, module in module.named_modules():
            if any(layer_name in name for layer_name in layer_names) and isinstance(module, Modular):
              print(f'[{name}] {key} activated')
              returns += ((name , getattr(module, key)(**config)), )

        return returns

    @staticmethod
    def get_attribute(obj, query):
        keys = query.split('.')
        current_obj = obj
        for key in keys:
            try:
                key = int(key)
            except ValueError:
                pass
            if isinstance(current_obj, dict):
                current_obj = current_obj.get(key)
            elif isinstance(current_obj, list):
                if ':' in key:
                  slice_obj = slice(*map(int, key.split(':')))
                else:
                  slice_obj = int(key)

                current_obj = current_obj[slice_obj]

            else:
                # print(current_obj, key)
                current_obj = getattr(current_obj, key)
        return current_obj


    def get(self, layer_names=None, key=None, module=None, **karg):
        module = self.module if (module is None) else module
        assert (module is not None), "pass a module"

        Modular = karg.get('Modular', self.Modular)

        returns = ()
        for name, module in module.named_modules():
            if any(layer_name in name for layer_name in layer_names) and isinstance(module, Modular):
              att = self.get_attribute(module, key)

              returns += ((name , att), )

        return returns


    def trainable_params(self, non_zero=False, module=None):
        """
        Function to print the parameters of the model and whether they are set for training.

        Args:
        - module: The PyTorch module.
        - non_zero: If True, counts only non-zero parameters.

        Returns:
        - count: Total number of trainable parameters.
        - total: Total number of parameters.
        """
        module = self.module if (module is None) else module
        assert module is not None, "Please pass a module"

        print(f"{'####'*10}")
        print(f"{'####'*2}{' Parameters of the Model: ':^5}{'######'}")
        print(f"{'####'*10}")
        count = 0
        total = 0

        print(module)
        for name, param in module.named_parameters():
            if non_zero:
                temp = torch.count_nonzero(param).item()
            else:
                temp = param.numel()

            total += temp
            if param.requires_grad:
              count += temp
              print(f"{name}: Trainable")
            # else:
              # print(f"{name}: No")

        print()
        print(f"Total not trainable: {total-count}")
        print(f"Trainable parameters {count} ({count / total:.2%}) {'non zero ' if non_zero else ' '}")
        print(f"{'####'*10}")

        return count, total


    def copy_arch(self, module=None):
      module = self.module if (module is None) else module
      assert (module is not None), "pass a module"

      def copy_with_shared_weights(model1):
          model2 = copy.deepcopy(model1)
          for name, param in model1.named_parameters():
              model2.get_parameter(name).data = param.data
          return model2

      return copy_with_shared_weights(module)


#######


class ModularUtils_class:
    def __init__(self, **kargs):
        # print(test)
        # Looping through named modules and assigning lambda functions
        for name, module in self.named_modules():
            # Creating a lambda function that calls custom with the module's name
            ModularUtils(module=module,
                                       tag=name, **kargs)
            # print(module.util.module)




## ⛱ Adapter

In [ ]:

def create_sub_dict(original_dict, keys_to_keep):
    """
    Create a sub-dictionary from the original dictionary by keeping only the specified keys.

    Args:
    - original_dict (dict): The original dictionary.
    - keys_to_keep (list): A list of keys to keep in the sub-dictionary.

    Returns:
    - sub_dict (dict): The sub-dictionary containing only the specified keys.
    """
    sub_dict = {key: original_dict[key] for key in keys_to_keep if key in original_dict}
    return sub_dict



import torch.nn as nn
import torch
import copy
from collections import OrderedDict


class Lora(nn.Module):
    def __init__(self, base_linear,
                 alpha= 0.5, r = 1,
                 **config):
        super(Lora, self).__init__()

        self.merged = False
        self.config = {
            'alpha': alpha,
            'r': r,
            **config  # Merge additional configuration options
        }
        # dont save as a dict
        self.base_linear = base_linear

        self.const = nn.Parameter(torch.Tensor([self.config.get("alpha")/self.config.get("r")]), requires_grad=False)
        self.config['o'], self.config['i'] = base_linear.weight.shape

        self.init()

    def init(self):
        w = self.base_linear.weight.detach().T

        w1 = nn.Linear(self.config["i"],
                       self.config["r"], bias=False, dtype=w.dtype)
        w1=w1.weight.T

        self.loraA = nn.Parameter(w1)


        self.loraB = nn.Parameter(torch.zeros(
            (self.config["r"], self.config["o"]), dtype=w.dtype)
        )




    def forward(self, x):

        return self.base_linear(x) + self.main(x)


    def main(self,x):
        low = ( (x @ self.loraA) @self.loraB )

        return self.const * low



#####################
    @torch.no_grad()
    def merge(self,name= None, arg=None):
        if name == None:
          name = '.'
        if self.merged == True:
          return print(f'[{name}]>> already Done')
        else:
          print(f'[{name}]>> Done!')

        dtype = self.loraB.dtype

        self.merged = True
        base_linear = self.base_linear

        base_linear.weight.data = base_linear.weight.to(dtype=dtype)

        base_linear.weight[:] += (self.const*self.loraA @ self.loraB).T


        # base_linear.weight[:] = (self.m/self.norm()) * base_linear.weight




    def export(self):
        keys_to_keep = [
            # 'const',
                        'loraA',
                        'loraB',
                        ]
        return create_sub_dict(dict(self.state_dict()),
                               keys_to_keep)

    @torch.no_grad()
    def unmerge(self, arg=None):
        if self.merged == True:
          return print(f'[{name}]>> already Done')
        else:
          print(f'[{name}]>> Done!')

        dtype = self.loraB.dtype

        self.merged = False
        base_linear = self.base_linear

        base_linear.weight.data = base_linear.weight.to(dtype=dtype)

        base_linear.weight[:] -= (self.const*self.loraA @ self.loraB).T

        # base_linear.weight[:] = (self.m/self.norm()) * base_linear.weight


# class MLora(nn.Module):
#   def __init__(self, base_linear, config):
#     super(MLora, self).__init__()

#   def forward(self, x):
#     return x



## 🕸 sparcity





https://pytorch.org/tutorials/prototype/semi_structured_sparse.html

https://github.com/locuslab/wanda?tab=readme-ov-file

https://github.com/VainF/Torch-Pruning?tab=readme-ov-file

https://eric-mingjie.github.io/wanda/home.html

In [ ]:
# from transformers import AutoTokenizer, AutoModel
# model_name = 'sentence-transformers/paraphrase-MiniLM-L3-v2'
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'
# modelref_s = AutoModel.from_pretrained(model_name)


from einops import einsum,pack

import torch.nn as nn
import torch
import copy
from collections import OrderedDict



def norm_d(tensor, p=2, dim=0, sum_old=None):
    sum_tensor = tensor.pow(p).sum(dim=dim)

    # If sum_old is provided, accumulate the sum
    if sum_old is not None:
        sum_tensor += sum_old

    return sum_tensor




# # W: weight matrix (C_out, C_in);
# # X: input matrix (N * L, C_in);
# # s: desired sparsity, between 0 and 1;

# # wanda

# def prune(W, X, s):
#     metric = W.abs() * X.norm(p=2, dim=0)   # get the Wanda pruning metric

#     _, sorted_idx = torch.sort(metric, dim=1)   # sort the weights per output
#     pruned_idx = sorted_idx[:,:int(C_in * s)]   # get the indices of the weights to be pruned
#     W.scatter_(dim=1, index=pruned_idx, src=0)   # get the index of pruned weights

#     return W


class Prune(nn.Module):
    def __init__(self, base_linear,
                s=0.8,acum=False,copyw=False,
                 **config):
        super(Prune, self).__init__()
        self.config = {
                's': s,
                'acum':acum,
                'copyw':copyw,
                **config
            }

        if self.config['copyw']:
          self.main = copy.deepcopy(base_linear)
        else:
          self.main = base_linear
        self.quantized = False


        self.sum_old = 0

        self.mask = nn.Parameter(torch.tensor([1], dtype=torch.bool),
                                 requires_grad=False)

        self.pruned = False


    @torch.no_grad()
    def forward(self, x):
        W = self.main.weight.detach()
        y = self.main(x)
        y_hat = self.filter_prune(W,x)
        # y_hat = self.filter_prune(W,x)

        # print(self.sum_old)
        print(torch.mean((y - y_hat)**2))
        return y_hat

    # def fake_prune(self, X):
    #     W = self.main.weight.detach()
    #     return  einsum((W*self.mask), X, "out inn, b l inn -> b l out")


    def prune(self):
      W = self.main.weight.detach()

      if self.pruned:
        print("already pruned!")
      else:
        self.pruned = True
        mask = self.mask

        if mask.dtype is not torch.bool:
          mask = mask.to(torch.bool)

        W[~mask] = 0

      # self.main = self.main

    def filter_prune(self, W, X):
        s = self.config['s']
        W = self.main.weight.detach()

        C_in = W.shape[1]
        X_join = pack(X, '* d')

        if self.config.get('acum'):
          self.sum_old = norm_d(X_join[0],
                                p=2, dim= 0,
                                sum_old= self.sum_old)
          norm_vec = self.sum_old.pow(1/2)
        else:
          norm_vec = X_join[0].norm(p=2, dim=0)


        metric = W.abs() * norm_vec    # get the Wanda pruning metric
        _, sorted_idx = torch.sort(metric, dim=1)   # sort the weights per output
        pruned_idx = sorted_idx[:,:int(C_in * s)]   # get the indices of the weights to be pruned
        self.pruned_idx = pruned_idx

        mask = torch.ones_like(W, dtype=W.dtype)

        mask.scatter_(dim=1, index=self.pruned_idx,
                      src=torch.zeros_like(self.pruned_idx, dtype=W.dtype))


        self.mask.data = mask

        return X @ (W*mask).T


# Mod_prune = ModularUtils(Modular=Prune,
#                          config_Mod= {
#                              's': 0.8,
#                              'acum':True
#                          })

# model_base = copy.deepcopy(modelref_s)
# model_mod = Mod_prune.copy_arch(model_base)


# base_w = model_base.get_submodule('encoder.layer')[-1].output.dense
# base_w, base_w.weight

# a = Mod_prune.app_Modular(
#     model_mod.get_submodule('encoder.layer')[-1].output,
#     ['dense']
# )

# mod_w = model_mod.get_submodule('encoder.layer')[-1].output.dense
# mod_w, mod_w.base_linear.weight

# model_mod(**tokenize_batch_s([
#     'hi',
#     'hi',
#     'hi',
#     'hi2'
#                                ]))

# # Mod_prune.activate_layers(
# #     module = model_mod,
# #     layer_names = ['dense'],
# #     activation_func='prune')

# # a[0].dense.pruned_idx

# # a[0].dense.prune()



#### prune + Lora

In [ ]:
# # faz um foward e progapaga o input com a mascara para guarda em todas as
# # layers com Prune, os indexs para transformar os valores em zero.


# class ModMix(nn.Module):
#     def __init__(self, base_linear, config):
#         super(ModMix, self).__init__()
#         self.Mix = nn.ModuleDict({
#             'base_linear': Prune(base_linear, config)
#         })

#         self.name = 'adapter_a'
#         self.Mix[self.name] = Lora(base_linear, config)
#         # self.Mix

#     def forward(self, x):
#       return self.Mix[self.name](x)

#     def set(self, name):
#       self.name = name


## 🔰 quantization

https://pytorch.org/tutorials/intermediate/dynamic_quantization_bert_tutorial.html

https://pytorch.org/tutorials/recipes/quantization.html

https://pytorch.org/docs/stable/quantization.html

> https://github.com/SqueezeAILab/SqueezeLLM#from-scratch-quantization

In [ ]:
# import onnx
# from onnxruntime.quantization import quantize_dynamic, QuantType

# model_fp32 = 'path/to/the/model.onnx'
# model_quant = 'path/to/the/model.quant.onnx'
# quantized_model = quantize_dynamic(model_fp32, model_quant)


In [ ]:
# from transformers import AutoTokenizer, AutoModel

# model_name = 'sentence-transformers/paraphrase-MiniLM-L3-v2'
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'
# modelref_s = AutoModel.from_pretrained(model_name)

import copy
def quantize_tensor(x, qmin, qmax, bits, symmetric=True, z=None):

    bits_n = 2 ** bits - 1
    s = (qmax - qmin) / bits_n

    if symmetric:
        if z is None:
            z = torch.round(torch.mean(x))
        xc = x
        q = int(bits_n / 2)
        qx = torch.clamp(torch.round(xc / s) - z, -q, q + 1)
    else:
        xc = (x - qmin) / (qmax - qmin)
        z = 0 if z is None else z
        qx = torch.clamp(torch.round(xc / s) - z, 0, bits_n)

    return qx, s




def fake_quant(W, dim=None, mask=None, bits=8, symmetric=True):
  z = 0

  if dim is None:
    qmin = torch.min(W)
    qmax = torch.max(W)
  else:
    qmin = torch.min(W, dim, keepdims=True).values
    qmax = torch.max(W, dim, keepdims=True).values

  qW, s = quantize_tensor(W,
                            qmin, qmax,
                            bits=bits,
                            symmetric= symmetric,
                            z = z)

  print(torch.mean((W - (qW * s))**2))
  print('done!')
  return qW, (s, qmin, qmax)




#############################




class QLinear(nn.Module):
    def __init__(self, qmin, qmax,
                 bits, s, W, z=None, mask=None):
        super(QLinear, self).__init__()
        self.bits = bits
        self.qmin = nn.Parameter(qmin, requires_grad=False)
        self.qmax = nn.Parameter(qmax, requires_grad=False)
        self.s = nn.Parameter(s, requires_grad=False)
        self.z = z
        self.W = W

    def forward(self, x):
      return self.W(x)

    def __repr__(self):
        qshape = list(self.qmin.data.shape)
        sshape = list(self.s.data.shape)
        if len(sshape)==0:
          qshape = sshape = [1]

        if (self.z is not None):
          zshape = list(self.z.data.shape)
          return f'''QLinear(W={self.W},\n\tbits={self.bits}, qmin_max={qshape}, z={zshape}, s={sshape})'''
        else:
          return f'''QLinear(W={self.W},\n\tbits={self.bits}, qmin_max={qshape}, s={sshape})'''

    @torch.no_grad()
    def export(self, **args):
      # self.W.weight.detach()
      self.W.weight.requires_grad = False

      if self.W.bias is not None:
        self.W.bias.data = self.W.bias.to(torch.float16)

      self.W.weight.requires_grad = False

      self.W.weight.data[:] = self.W.weight.data / self.s.data
      w = self.W.weight.detach().to(dtype=torch.int8)

      self.W.weight.data = w

      self.s.data = self.s.to(dtype=torch.float16)
      self.qmin.data = self.qmin.to(dtype=torch.float16)
      self.qmax.data = self.qmax.to(dtype=torch.float16)


##################################




class Quantize_Linear(nn.Module):
    def __init__(self, base_linear,
                 bits = 8, acum= False, symmetric= True, copyw=False,
                 **config):
        super(Quantize_Linear, self).__init__()

        self.config = {
                'bits': bits,
                'acum':acum,
                "symmetric": symmetric,
                "copyw":copyw,
                **config
            }
        # print(self.config)

        # print(config)

        self.merged = False
        if self.config['copyw']:
          self.main = copy.deepcopy(base_linear)
        else:
          self.main = base_linear
        self.quantized = False

    def forward(self, x):
      return self.main(x)


    def quantize(self, dim=None, mask=None, **kargs):
        # kargs
        # print(self.config.get('bits'))

        if self.quantized:
          print("already quantized!")
        else:
          self.quantized = True
          linear = self.main
          bits = self.config.get('bits')
          symmetric = self.config.get('symmetric')
          W=linear.weight.detach()
          dim = 1 if dim is None else dim

          if mask is not None:

            ds = W.shape[dim]

            l = [1]*ds
            qmins = []
            qmaxs = []
            ss = []

            wm = zip(W.split(l,dim), mask.split(l,dim))


            for ww, m in wm:
              ws = ww[m]
              if len(ws)>1:
                wq, (s, qmin, qmax) = fake_quant(ws,0, bits=bits, symmetric=symmetric)
              else:
                if len(ws)==0:
                  ws = torch.tensor([0])
                  s = torch.tensor([1])
                  qmin = qmax = ws
                else:
                  qmin = s = qmax = ws
                wq = 1

              ww[m] = wq *s
              ww[~m] = 0

              qmins.append(qmin)
              qmaxs.append(qmax)
              ss.append(s)

            qmin = torch.cat(qmins)
            qmax = torch.cat(qmaxs)
            s = torch.cat(ss)

            layer = QLinear(qmin = qmin, qmax = qmax,
                     bits = bits, s = s, W=linear, mask=True)

          else:
            w, (s, qmin, qmax) = fake_quant(W, dim = dim, bits=bits, symmetric=symmetric)
            W[:] = w*s
            layer = QLinear(qmin = qmin, qmax = qmax,
                      bits = bits, s = s, W=linear)

          self.main = layer



#### quantization + lora

In [ ]:
# from einops import einsum
# from einops import pack


# import torch.nn as nn
# import torch
# import copy
# from collections import OrderedDict


# # faz um foward e progapaga o input com a mascara para guarda em todas as
# # layers com Prune, os indexs para transformar os valores em zero.


# class ModMix(nn.Module):
#     def __init__(self, base_linear, config):
#         super(ModMix, self).__init__()
#         self.Mix = nn.ModuleDict({
#             'base_linear': Quantize_Linear(base_linear, config)
#         })

#         self.name = 'adapter_a'
#         self.Mix[self.name] = Lora(self.Mix['base_linear'].qlinear.W, config)
#         # self.Mix

#     def forward(self, x):
#       return self.Mix[self.name](x)

#     def set(self, name):
#       self.name = name




# def freeze_model(model):
#     for param in model.parameters():
#         param.requires_grad = False

# freeze_model(modelref_s)


# Mod_quant = ModularUtils(Modular=ModMix,
#                          config_Mod= {
#                              'bits': 3,
#                              'alpha':0.2,
#                              'r':2,
#                              'call':True
#                             #  'acum':True
#                          })

# model_base = copy.deepcopy(modelref_s)
# model_mod = Mod_quant.copy_arch(model_base)


# sub = model_mod.get_submodule('encoder.layer')[-1]
# a = Mod_quant.app_Modular(sub,['output.dense'])

# a[0]

In [ ]:

# Some trys

# import torch

# # Algorithm 1: LoftQ
# # input Pre-trained weight W, target rank r, N-bit quantization function qN(.), alternating step T

# def LoftQ(W, r, qN, T):
#     # Initialize
#     A = torch.zeros(W.size(0), r)
#     B = torch.zeros(r, W.size(1))

#     for t in range(T):
#         # Obtain quantized weight
#         Qt = qN(W - torch.matmul(A, B))

#         # Obtain low-rank approximation
#         U, S, V = torch.svd(W - Qt)
#         At = U[:, :r]
#         Bt = torch.matmul(torch.diag(S[:r]), V[:r, :])

#         # Update A and B
#         A = At
#         B = Bt

#     return Qt, At, Bt

# # Usage
# # Assuming W is your pre-trained weight tensor
# # r is your target rank
# # qN is your N-bit quantization function
# # T is the number of alternating steps

# # Qt, At, Bt = LoftQ(W, r, qN, T)


# import torch

# #unifom
# def quantization(x, bits=8):
#     # Assuming x is a PyTorch tensor
#     # Quantize the input tensor to a specified number of bits

#     # Determine the quantization range based on the number of bits
#     qnt_max = torch.max(x)
#     qnt_min = torch.min(x)
#     bits = 2 ** bits - 1

#     s = (qnt_max-qnt_max)/bits

#     # Scale the input tensor to fit within the quantization range
#     x_scaled = torch.clamp(x * s, quant_min, quant_max)

#     # Round the scaled tensor to the nearest integer
#     x_rounded = torch.round(x_scaled)

#     # Scale back to the original range
#     x_quantized = x_rounded / quant_max

#     return x_quantized

# # Example usage
# # Assuming x is your input tensor and bits is the number of bits for quantization
# # x_quantized = quantization(x, bits)

# `

# import torch
# import torch.nn as nn

# import bitsandbytes as bnb
# from bnb.nn import Linear4bit

# fp16_model = nn.Sequential(
#     nn.Linear(64, 64),
#     nn.Linear(64, 64)
# )

# quantized_model = nn.Sequential(
#     Linear4bit(64, 64),
#     Linear4bit(64, 64)
# )

# quantized_model.load_state_dict(fp16_model.state_dict())
# quantized_model = quantized_model.to(0) # Quantization happens here

## 🗿 MIX

In [ ]:
class ModMix(nn.Module):
    def __init__(self, base_linear, **config):
        super(ModMix, self).__init__()
        self.config = config

        self.tag = 'base_linear'
        self.main_tag = 'Base'
        self.name_adapt = 'main'
        self.merged = False

        self.Mix = nn.ModuleDict({
            'base_linear': base_linear
        })

        self.main = self.Mix['base_linear']

        self.set_dict = {
            'Prune': lambda base_linear, config: Prune(base_linear,
                                               **config),
            'Quant': lambda base_linear, config: Quantize_Linear(base_linear,**config),
            'Adapt': {
                'new': lambda base_linear, config: Lora(base_linear,
                                                **config),
                'available': dict()
                }
            }


    def forward(self, x):
      return self.main(x)

    def merge(self):
      if self.merged:
        return print('Already Done!')

      main = []
      mask = None
      self.mask = mask
      self.main_tag = 'base_linear'

      self.Mix['base_linear']
      names = list(self.Mix.keys() )

      for name in names:

        if name == 'Prune':
          self.mask = copy.deepcopy(self.Mix['Prune'].mask)
          mask = self.mask.data.to(torch.bool)
          del self.Mix['Prune']

        elif name == 'Quant':
          self.Mix['Quant'].quantize(1, mask)
          self.Mix['base_linear'] = copy.deepcopy(self.Mix['Quant'].main)
          del self.Mix['Quant']
          del self.mask

      self.merged  = True
      self.main = self.Mix['base_linear']
      self.tag = 'pruned+quantized'


#####################################################

    def set(self, name, **config):
      base = config.get('base', 'base_linear')
      if base == "Prune":
        base_linear = self.Mix[base].main
      else:
        base_linear = self.Mix[base]
      set_value = self.set_dict.get(name, None)

      config = self.config if config is None else config
      # print(config)


      if self.merged:
        base_linear = base_linear.W
        self.tag = 'pruned+quantized'
        self.main_tag = 'base_linear'



      if name == "Prune":
        # print(config)
        if config.get("fake_prune", False):
          # print("config")
          # print(config)
          self.main = self.Mix["Prune"].main
        else:
          self.Mix["Prune"] = set_value(base_linear, config)
          self.Mix["Prune"].main  = base_linear
          self.main = self.Mix["Prune"]


        self.tag+= '+prune'
        self.main_tag = 'Prune'


      elif name == "Quant":
        self.Mix["Quant"] = set_value(base_linear, config)
        self.main = self.Mix["Quant"]
        self.tag+= '+quant'
        self.main_tag = 'Quant'

      elif name == "Base":
        self.main = self.Mix["base_linear"]
        self.main_tag = 'base_linear'


      elif name == "Adapt":

        self.set_dict['Adapt']['available'][name] = self.set_dict['Adapt']['new'](base_linear, config)

        self.set_dict['Adapt']['active'] = self.set_dict['Adapt']['available'][name]
        self.Mix["Adapt"] = self.set_dict['Adapt']['active']
        self.name_adapt = name

        self.set_dict['Adapt']['active'].base_linear = base_linear
        self.main = self.set_dict['Adapt']['active']

        ver = self.tag.split('+')
        if "adapt" not in  ver[-1]:
          self.tag+= '+adapt'
        self.main_tag = 'Adapt'


      else:
        if (name not in self.set_dict['Adapt']['available'].keys()):
          self.set_dict['Adapt']['available'][name] = self.set_dict['Adapt']['new'](base_linear, config)

        self.set_dict['Adapt']['active'] = self.set_dict['Adapt']['available'][name]
        self.Mix["Adapt"] = self.set_dict['Adapt']['active']
        # self.main = self.Mix["Adapt"]
        # print(self.set_dict['Adapt']['active'].main)

        self.set_dict['Adapt']['active'].base_linear = base_linear
        self.main = self.set_dict['Adapt']['active']

        self.name_adapt = name

        ver = self.tag.split('+')
        if f"adapt[{name}]" not in  ver[-1]:
          self.tag+= f'+adapt[{name}]'
        self.main_tag = 'Adapt'


    def del_layer(self, name, base):
      self.Mix[base] = copy.deepcopy(self.Mix[base])
      self.main = self.Mix[base].main

      if isinstance(name, list):
        for n in name:
          del self.Mix[n]
      else:
        del self.Mix[name]


#####################################################

    def __repr__(self):
      # return self.main.__repr__()
        p = self.Mix.__repr__()
        p= p.replace('\n ','\n\t  ')
        p= p.replace('\n)','\n\t)')
        p = re.sub('(Adapt\w?\))',f'\\1 [{self.name_adapt}]', p)

        return f'''Active(main={self.main_tag}\n\ttag={self.tag},\n\tMix={p}\n)'''

## examples

### ex1

In [ ]:


import torch
from torch import nn
import copy
import re

lin = nn.Linear(5,10, True)

d = ModMix(lin)

# d.set('test')
# d(x)

# d.main.init()

# d.main.merge()

# print(d.Mix['base_linear'].weight)

# d.set('Quant')
# # d.merge()
# d(x)

# d.main.quantize(0)

# d.main.main.W.weight


d.set('Prune')
x = torch.rand((2,1,5))

d(x)
d.main.prune()

d, d.main.mask

d.main.mask.data


# print(d.Mix['base_linear'].weight)
# d
# print(d.main)
# # print(d.Mix['base_linear'].W.weight)

# d.set('Base')

# d

### ex2

In [ ]:
class subsub(nn.Module):
    def __init__(self):
        super(subsub, self).__init__()
        self.a = nn.Linear(2, 3)
        self.b = nn.Linear(3, 3)
        self.sss = nn.Sequential(
            nn.Linear(3, 4),
            nn.Linear(4, 5),
            self.b,
        )
        self.ff = nn.Linear(5, 2)


class ComplexModel(nn.Module,
                  ModularUtils_class
                   ):
    def __init__(self):
        super(ComplexModel, self).__init__()


        self.layer1 = nn.Linear(2, 3)
        self.layer2 = nn.Linear(3, 3)
        self.submodule = nn.Sequential(
            nn.Linear(3, 4),
            nn.Linear(4, 5),
            self.layer2,
        )
        self.deep = nn.ModuleList([
            subsub(),
            subsub()
        ]
        )
        self.layer3 = nn.Linear(5, 2)

        ModularUtils_class.__init__(self,
                                    Modular = Lora)  # Manually call enhance modules class constructor

#######



# Mod = ModularUtils(Modular=Lora)
model = ComplexModel()

model.layer1.util.module

In [ ]:
# Mod = ModularUtils(Modular=Lora)
named = lambda n: print(f"{'#'*50}\n{'# '*5}{n:^20} #\n{'#'*50}")

model = ComplexModel()


model_base = copy.deepcopy(model)
model_mod = model.util.copy_arch(model_base)


sub = model_mod.get_submodule("deep")

k, n = sub.util.map(
    name_module = ["a", 'ff'],
    config_Mod={'alpha':10}
)

# k.util.module
k.util.trainable_params(True)
named('lora_module')
print()
print(n[0])
print(k)


named('model_base')
print(model_base)

named('model_mod')
print(model_mod)

# # Export weights
all_weights = model_mod.util.export_mod( join= False)

# # Load weights back
model_mod.util.load_mod(all_weights)

# all_weights = model_mod.util.export_mod( join= True)
# a = model_mod.load_state_dict(all_weights, strict=False)
# a.missing_keys



ref = k[0].a
# ref = k.deep[0].a
print("#########################")


named("k[0].a.base_linear.weight")
print(ref.base_linear.weight)

ref.merge()
# k[0].a.unmerge()

assert torch.allclose(
    ref.base_linear.weight,
    model_base.deep[0].a.weight) , "model_base.deep[0].a.weight differ"

assert torch.allclose(
    ref.base_linear.weight,
    model_mod.deep[0].a.base_linear.weight), "model_mod.deep[0].a.base_linear.weight differ"

# model_base.deep[0].a.weight.data[:] = 0

# ref.base_linear['linear'].weight.data[:] = 0

# print(ref.base_linear['linear'].weight)

named("model_base.deep[0].a.weight")
print(model_base.deep[0].a.weight)

In [ ]:
model = ComplexModel()

model_base = copy.deepcopy(model)
model_mod = model.util.copy_arch(model_base)


sub = model_mod.get_submodule("deep")

k, n = sub.util.map(
    name_module = ["a", 'ff'], share_weights=False
)


print("######"*10)
print("## lora_module ##")
print(n)
print(k)

print("######"*10)
print("## model_base ##")
# print("\nModel ori:")
print(model_base)

print("######"*10)
print("## model_mod ##")
# print("\nModel ori:")
print(model_mod)

# # Export weights
# model.util.module

all_weights = k.util.export_mod(join= False)

# # Load weights back
k.util.load_mod(all_weights, model)


ref = k[0].a
# ref = k.deep[0].a
print("#########################")
print("k[0].a.base_linear.weight")
print(ref.base_linear.weight)
ref.merge()
# k[0].a.unmerge()
print("------- k[0].a.base_linear.weight -------")
print(ref.base_linear.weight)

print("------- model_base.deep[0].a.weight -------")
print(model_base.deep[0].a.weight)


### bert example

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

# model

In [ ]:
# Mod = ModularUtils()


# # model = ComplexModel()

# Mod.app_Modular(
#     model, # Modular
#                 ["query", 'key'] # linear layes (type_layers)
#                 )


# model.get_submodule('bert.encoder')

# 🧠 Model Arch

### util functions


In [ ]:

# Example usage:
# file_path = "/content/content/slr_data_tcc/synergy/Smid_2020.csv"
# df = preprocess_csv_file(file_path)


# df_train = df.groupby('label').sample(4)
# df_temp = df[~df.index.isin(df_train.index)]

# df_eval = df_temp.groupby('label').sample(2)
# df_test = df_temp[~df_temp.index.isin(df_eval.index)]

# df['label'].value_counts()

# print(df_train.iloc[0])
# df_train.iloc[0]['text']



from einops import unpack
from itertools import chain
from collections import namedtuple

from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import Dataset, DataLoader
from collections import namedtuple
import os
# max_seq_length = 512


# tokenizer_t_ref = AutoTokenizer.from_pretrained('allenai/specter2_base')
# tokenize_batch_t = lambda train_text: tokenizer_t_ref.batch_encode_plus(
#     train_text,
#     add_special_tokens=True,
#     padding='max_length',
#     max_length=max_seq_length,
#     truncation=True,
#     return_tensors='pt',
#     return_attention_mask=True
# )



InputExample = namedtuple('InputExample', ['texts', 'label'])


class SIMDataset(Dataset):
    def __init__(self, data, tokenizer,
                 n_pairs = 0, n_pairs_random= False,
                 emb =None, device = "cuda", b = 16, pttn = '* d', dir=None,
                 map = {"x": "text", "y": "label"}):

        self.b = b
        self.device = device
        self.tokenizer = tokenizer
        self.emb = emb
        self.n_pairs = n_pairs
        self.squeeze = lambda input: {x: y.squeeze() for x, y in input.items()}
        self.pttn = pttn

        x = data[map.get("x")].to_numpy()
        y = data[map.get("y")].to_numpy()
        self.is_list = isinstance(x[0], list) or (n_pairs > 0)

        # if emb:
        #   x=x.tolist()
        #   load = DataLoader(x, batch_size=b, shuffle=False)
        #   with torch.no_grad(), tqdm(total=len(load), desc='Processing Batches') as pbar:
        #     all = []
        #     for batch_idx, d in enumerate(load):
        #       d = self.tokenizer(d)
        #       d = Trainer.process_batch(d, device = device)
        #       d = self.emb(**d)
        #       all.append(d)

        #   x = torch.concat(all)
        #   x = Trainer.process_batch(x, device = "cpu")


        if n_pairs>0:
          pairs = []
          if not n_pairs_random:
            for _ in range(n_pairs):
                pairs = self.sentence_pairs_generation(x, y, pairs)
          else:
            pairs = self.generate_random_pairs(x, n_pairs)


          self.examples = pairs
        else:
          self.examples = tuple(zip(x,y))

        # print("len(self.examples)")
        # print(len(self.examples))

        if emb:
          self.examples = self.embed(emb,self.examples, dir, b)



    def embed(self, emd, examples ,dir, b=18):
      def pipe(x):
        x = self.tokenizer(x)
        x = Trainer.process_batch(x, device = self.device)
        return x

      # self.pttn = '* b a'
      if dir is not None:
        os.makedirs(dir, exist_ok=True)

      load = DataLoader(examples, batch_size=b, shuffle=False)

      # emd.to(self.device)
      with torch.no_grad(), tqdm(total=len(load), desc='Processing Batches') as pbar:
        all = []
        for batch_idx, d in enumerate(load):
          x = d[0]
          y = d[1]

          if self.is_list:
            x = [emd(**pipe(x_in)).to('cpu') for x_in in x]
            x_k = [unpack(x_in, [[1]]*len(y), self.pttn) for x_in in x]
            ex = tuple(zip(zip(*x_k), y))
          else:
            x = emd(**pipe(x))
            ex = tuple(zip(x,y))

          # Save batch data to file
          if dir is not None:
            batch_filename = os.path.join(dir, f'batch_{batch_idx}.pt')
            torch.save(ex, batch_filename)
          else:
            all.append(ex)

          x = Trainer.process_batch(x, device ="cpu")
          pbar.update(1)

        if dir is not None:
          del examples
        else:
          if len(all) > 1:
            all = list(chain(*all))
          else:
            all = all[0]
          examples = all

      torch.cuda.empty_cache()
      return examples

    def generate_random_pairs(self, texts, num_pairs=1):
        pairs = []
        num_texts = len(texts)

        if num_pairs == 1:
            # Generate one pair for each text with every other text
            for i in range(num_texts):
                for j in range(i + 1, num_texts):
                    pairs.append(InputExample(texts=(texts[i], texts[j]), label=-1))
        else:
            # Generate exactly num_pairs random pairs
            for _ in range(num_pairs):
                pair_indices = np.random.choice(num_texts, size=2, replace=False)
                pairs.append(InputExample(texts=(texts[pair_indices[0]], texts[pair_indices[1]]), label=-1))
        return pairs



    def sentence_pairs_generation(self, sentences, labels, pairs):

        # Initialize two empty lists to hold the (sentence, sentence) pairs and
        # labels to indicate if a pair is positive or negative

        num_classes = np.unique(labels)
        label_to_idx = {x: i for i, x in enumerate(num_classes)}
        positive_idxs = [np.where(labels == i)[0] for i in num_classes]
        negative_idxs = [np.where(labels != i)[0] for i in num_classes]

        for first_idx in range(len(sentences)):
            current_sentence = sentences[first_idx]
            label = labels[first_idx]
            second_idx = np.random.choice(positive_idxs[label_to_idx[label]])
            positive_sentence = sentences[second_idx]
            # Prepare a positive pair and update the sentences and labels
            # lists, respectively
            pairs.append(InputExample(texts=[current_sentence, positive_sentence],
                                      label=torch.tensor([1], dtype= torch.float32)))

            third_idx = np.random.choice(negative_idxs[label_to_idx[label]])
            negative_sentence = sentences[third_idx]
            # Prepare a negative pair of sentences and update our lists
            pairs.append(InputExample(texts=[current_sentence, negative_sentence],
                                      label=torch.tensor([0], dtype= torch.float32)))
        # Return a 2-tuple of our sentence pairs and labels
        return pairs

    def get_slices(self, some_tuple, indices):
      return tuple(some_tuple[i] for i in indices)


    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        # batch = self.get_slices(self.examples,idx)
        # print(type(idx))
        # print(idx)
        batch = self.examples[idx]
        x,y = batch

        if self.emb:
          if self.n_pairs > 0:
            return [x_in.squeeze() for x_in in x], y
          else:
            if isinstance(x, (list, tuple)):
              return [x_in.squeeze() for x_in in x], y
            else:
              return x.squeeze(), y
        else:
          if self.n_pairs > 0:
            if isinstance(x[0], (list, tuple)):
              return [self.squeeze(self.tokenizer(x_in)) for x_in in x], y
            else:
              return [self.squeeze(self.tokenizer([x_in])) for x_in in x], y
          else:
            if isinstance(x, (list, tuple)):
              return [self.squeeze(self.tokenizer([x_in])) for x_in in x], y
            else:
              return self.squeeze(self.tokenizer([x])), torch.Tensor([y])



class LazyLoadDataset(Dataset):
    def __init__(self, data_paths):
        self.data_paths = data_paths

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        # Load data at idx only when it's requested
        tensor_path = self.data_paths[idx]
        tensor = torch.load(tensor_path)
        return tensor





import torch
from torch.utils.data import Dataset, DataLoader
from collections import namedtuple

# Define a namedtuple for InputExample
InputExample = namedtuple('InputExample', ['texts', 'label'])

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        return sample.texts, sample.label

# Define some sample data using InputExample
sample_data = [InputExample(texts=torch.randn(3, 32, 32), label=torch.randint(0, 10, (1,)).item()) for _ in range(100)]

# Create an instance of the custom dataset
custom_dataset = CustomDataset(sample_data)

# Create a DataLoader to iterate over the dataset
batch_size = 10
shuffle = True
num_workers = 4
data_loader = DataLoader(dataset=custom_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

# Iterate through the DataLoader
for batch_idx, (texts, labels) in enumerate(data_loader):
    # texts and labels represent a batch of data
    print(f"Batch {batch_idx}: Texts shape: {texts.shape}, Labels shape: {labels.shape}")


# sim_data = SIMDataset(df_train, tokenize_batch_t, 2)
# train_dataloader = DataLoader(sim_data,
#                               shuffle=True,
#                               batch_size=batch_size)
# for x in train_dataloader:
#   break




### Similarity

In [ ]:
import numpy as np
from torch import nn
from collections import namedtuple


class SentenceEmbedding(nn.Module):
    def __init__(self,
                 model,
                 normalize=False,
                 pooling = False,
                 head = None,
                 **kargs):
        super(SentenceEmbedding, self).__init__()

        self.model = model
        hidden = model.config.hidden_size
        self.normalize = normalize
        self.pooling = pooling
        self.with_head = kargs.get('with_head', False)
        self.is_bert = kargs.get('with_head', False)


        self.head =  nn.Sequential(
            # nn.GELU(),
            nn.Linear(hidden, 384),
            # nn.GELU(),
            # nn.Linear(768, 256),
            # nn.GELU(),
            # nn.Linear(256, 256)
        ) if (head is None) and self.with_head else head


    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

        embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

        return embeddings

    def __getattr__(self, name: str):
        """Forward missing attributes to the wrapped module."""
        try:
            return super().__getattr__(name)  # defer to nn.Module's logic
        except AttributeError:
            return getattr(self.model, name)



    def forward(self, x):

        if isinstance(x, torch.Tensor):
          embeddings = self.model(x)
        else:
          embeddings = self.model(**x)


        if self.pooling:
          embeddings = self.mean_pooling(embeddings,
                                         x.get("attention_mask") )
        else:
          embeddings = embeddings[0][:,0,:]


        if self.with_head:
          embeddings = self.head(embeddings)

        if self.normalize:
          embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)

        return embeddings


import torch
from torch import nn, Tensor
from typing import Iterable, Dict
from typing import List



class loss_sim(nn.Module):

  def __init__(self, model, layer_bert = False,
               tau = 0.1, device="cuda"
               ):
    super(loss_sim, self).__init__()
    self.model = model
    self.loss = torch.nn.CosineEmbeddingLoss(margin=0.25, reduction='mean')
    self.tau = torch.Tensor([tau]).to(device)
    self.loss1 = torch.nn.CosineSimilarity(dim=1)
    self.loss2 = nn.MSELoss()
    self.n_labels = 2
    self.loss_3 = nn.BCEWithLogitsLoss(reduction = 'mean',
                                    pos_weight=torch.FloatTensor([2.5]))
    # self.n_0 = torch.Tensor([0]).to('cuda')

    self.layer_bert = layer_bert

  def forward(self, texts, labels):

    embeddings = [self.model(emd) for emd in texts]
    labels = labels.squeeze()

    s = self.loss1(embeddings[0], embeddings[1])

    p = s[labels == 1]
    n = s[labels == 0]

    a = torch.mean(
        -torch.log(
            torch.exp(p/self.tau)/\
             (torch.sum(torch.exp(n/self.tau)))
        )
    )

    # cp = torch.mean((1-torch.clamp(
    #     p, min= 0.8
    #     )
    # ))


    # cn = torch.mean((1-torch.clamp(
    #     n, min= 0.8
    #     )
    # ))


    cp = torch.mean((1-p)**2)
    cn = torch.mean((-1-n)**2)
    return (0.3)*a+ (0.7)*(cp+cn)

    # return (0.6)*a+ (0.4)*(cp+cn)

### Classification

In [ ]:
# # Complete model
# class SLR_Classifier(nn.Module):
#   def __init__(self, **data):
#     super(SLR_Classifier, self).__init__()

#     # Pre-trained model
#     self.Encoder = data.get('model')

#     self.hidden_size_input = self.Encoder.config.hidden_size

#     self.hidden_size = 200


#     # Feature Map Layer
#     self.feature_map = nn.Sequential(
#             # nn.LayerNorm(self.Encoder.model.config.hidden_size),
#             # nn.BatchNorm1d(self.Encoder_config.hidden_size),
#             # nn.Dropout(data.get("drop", 0.5)),
#             nn.Linear(self.hidden_size_input, self.hidden_size),
#             nn.Tanh(),
#             nn.Linear(self.hidden_size, 1),
#             # nn.Tanh(),
#             # nn.Linear(200, 1)
#             # nn.Dropout(data.get("drop", 0.5)),
#         )


#     # Initializing layer parameters
#     # nn.init.normal_(self.feature_map[1].weight, mean=0, std=0.00001)
#     # nn.init.zeros_(self.feature_map[1].bias)

#   def __getattr__(self, name: str):
#       """Forward missing attributes to the wrapped module."""
#       try:
#           return super().__getattr__(name)  # defer to nn.Module's logic
#       except AttributeError:
#           return getattr(self.model, name)

#   # Feed forward
#   def forward(self, input_ids,
#               attention_mask,
#               token_type_ids,
#               train=True,
#               **kargs):

#     predict = self.Encoder(**{"input_ids":input_ids,
#                               "attention_mask":attention_mask,
#                               "token_type_ids":token_type_ids})
#     feature = self.feature_map(predict)
#     # logit = self.classifier(feature)
#     representations = torch.Tensor([1])
#     if train:
#       return feature
#     else:
#       return representations, feature




# from torch.utils.data import Dataset, DataLoader


# import torch
# from torch import nn, Tensor
# from typing import Iterable, Dict
# from typing import List


# class loss_slr(nn.Module):
#   def __init__(self, model):
#     super(loss_slr, self).__init__()
#     self.model = model
#     self.loss = nn.BCEWithLogitsLoss(reduction = 'mean',
#                                     pos_weight=torch.FloatTensor([2.5])
#                                     )


#   def forward(self, text, label, **kargs):
#     embeddings = self.model(**text)
#     return self.loss(embeddings.to(torch.float), label.to(torch.float))


## Distill

Criar codigo para que consiga independente da quantidade de textos, fazer o embeding e retornar da mesma forma.

```
[
  ((text1,...,), label),
  ...,
  ((text1,...,), label),
]
```

```
[
  ((emb1,...,), label),
  ...,
  ((emb1,...,), label),
]
```

deve ser squezaded o emb, pode ser passado o batch para agregar quantidade de exemplos q pode ser passado

In [ ]:
from torch.utils.data import DataLoader
from einops import pack, unpack

n = 50
ex = 2
input_data = torch.randn(n,ex, 5)
output_data = torch.randn(n, 1)
batch = 20


def batch_load(input, output, batch):
  input_data = input.get('data')
  output_data = output.get('data')

  input_cfg = input.get('config')
  output_cfg = output.get('config')


  full = input_data.shape[0]//batch
  batch_as = [[batch]]*full
  rest =  input_data.shape[0]%batch

  if rest > 0:
    batch_as += [[rest]]

  # input_data.shape
  in_a = unpack(input_data, batch_as, input_cfg)
  out_a = unpack(output_data, batch_as, output_cfg)


  return zip(in_a, out_a)







a = batch_load(
    input = {
        "data": input_data,
        "config": "* ex x"
    },
    output = {
        "data": output_data,
        "config": "* y"
    },
    batch = batch
)





In [ ]:
import torch.nn.functional as F

##### similatiry distill
# the model is trained to distill the similarity of the teacher

class few_shot_loss_distill(nn.Module):
  def __init__(self, model_s, model_t,
               t=0, loss=0):
    super(few_shot_loss_distill, self).__init__()
    self.model_t = model_t
    self.model_t.eval()
    self.model_s = model_s
    self.loss_choice = loss
    self.tau = 0.065
    self.t = t
    self.loss1 = torch.nn.CosineSimilarity(dim=1)
    self.distill = lambda x: F.log_softmax(x, dim = 0)

    self.loss = torch.nn.CosineEmbeddingLoss(margin=0.25, reduction='mean')
    # self.proj = nn.Sequential(
    #         # nn.GELU(),
    #         nn.Linear(384, 384),
    #         # nn.GELU(),
    #         # nn.Linear(384, 384)
    #         )


    # # return self.loss(embeddings[0],embeddings[1],labels)


  def forward(self, texts, labels):
    # print("texts")
    # global texts
    texts = tuple(texts)
    # print(texts)
    # embeddings_s = [self.model_s(**emd) for emd in texts]
    # embeddings_s2 = [self.proj(self.model_s(**emd)) for emd in texts]
    embeddings_s = [self.model_s(**emd) for emd in texts]

    # if self.model_t:
    embeddings_t = [self.model_t(**emd) for emd in texts]


    # embeddings_s = torch.concat(embeddings_s)
    # embeddings_t = torch.concat(embeddings_t)

    # sz = self.loss1(embeddings_s,embeddings_t)

    # embeddings = pack([embeddings_t, embeddings_s], '* b h')[0]
    # loss_kd =-self.distill(embeddings/self.tau)
    # return torch.mean(loss_kd)

    # print(sz.shape)
    # labels = labels.squeeze()
    # labels[labels == 0] = -1

    # p = sz[labels == 1]
    # n = sz[labels == -1]


    # return sz
    # return self.loss(embeddings[0],embeddings[1],labels)

    s_t = self.loss1(*embeddings_t)
    s_s = self.loss1(*embeddings_s)

    labels[labels == 0] = -1
    labels = labels.squeeze()

    p_t = s_t[labels == 1]
    n_t = s_t[labels == -1]

    p_s = s_s[labels == 1]
    n_s = s_s[labels == -1]

    s = (s_t-s_s)

    p = s[labels == 1]
    n = s[labels == -1]

    if self.loss_choice == 0:
      # s = s[s>]
      dist1 = torch.mean(
          -torch.log(
              torch.exp(s/self.tau)/\
              (torch.sum(torch.exp(s/self.tau)))
          )
      )

      return  dist1

    elif self.loss_choice == 1:
      dist = torch.mean(
          -torch.log(
              torch.exp(p/self.tau)/\
              (torch.sum(torch.exp(n/self.tau)))
          )
      )
      return  dist


    # dist_s = torch.mean(
    #     -torch.log(
    #         torch.exp(p_s/self.tau)/\
    #          (torch.sum(torch.exp(n_s/self.tau)))
    #     )
    # )

    # dist_t = torch.mean(
    #     -torch.log(
    #         torch.exp(p_t/self.tau)/\
    #          (torch.sum(torch.exp(n_t/self.tau)))
    #     )
    # )



    # return torch.mean((s_t-s_s)**2)



##### head distill
# make a new layer to distill the logits of the teacher

class loss_distill_to_light(nn.Module):
  def __init__(self, model_s, model_t = None,
               tau = 0.1, t=0):
    super(loss_distill_to_light, self).__init__()
    self.model_t = model_t
    self.model_s = model_s

    self.tau = tau
    self.t = t

    # (workers batch hidden)
    self.loss1 = torch.nn.CosineSimilarity(dim=1)
    self.distill = lambda x: F.log_softmax(x, dim = self.t)


  def forward(self, texts, labels):
    if self.model_t:
      embeddings_t = [self.model_t(**emd) for emd in texts]
    embeddings_s = [self.model_s(**emd) for emd in texts]

    embeddings = pack([embeddings_t, embeddings_s], '* b h')[0]

    loss_kd =-self.distill(embeddings/self.tau)

    return torch.mean(loss_kd, self.t)

    # s = self.loss1(embeddings_t, embeddings_s)

    # return torch.sum(s**2)



# import itertools
# a = itertools.tee(a,2)









# 📓 Train pipeline


## Class

In [ ]:
import gc



class Trainer:
    def __init__(self,
                 losses=None,
                 optimizer=None,
                 train_dataloader=None,
                 total_iterations=None,
                 val_dataloader = None,
                 lr=None,
                 accumulation_steps=None,
                 scheduler= None,
                 device = "cpu",
                 **kwargs):

        self.losses = losses
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.total_iterations = total_iterations
        self.lr = lr
        self.accumulation_steps = accumulation_steps
        self.optimizer = optimizer
        self.scheduler = scheduler


        self.device =device #= "cuda" if torch.cuda.is_available() else "cpu"

        self.loss_values = []



    @classmethod
    def process_batch(cls, batch, device = 'cuda', generator = True):

        def to_device_recursive(item):
            if isinstance(item, dict):
                return {key: to_device_recursive(value) for key, value in item.items()}
            elif isinstance(item, (list, tuple)):
                if generator:
                  return (to_device_recursive(value) for value in item)
                else:
                  return [to_device_recursive(value) for value in item]
            else:
                return item.to(device)

        batch = to_device_recursive(batch)
        return batch


    def compute_loss(self, batch):
        batch = self.process_batch(batch, self.device)
        # y = self.process_batch(batch[1], self.device)

        return self.losses(*batch)


    def gradient_update(self):
        if (self.iteration + 1) % self.accumulation_steps == 0:
            self.optimizer.step()
            if self.scheduler:
              self.scheduler.step()

            self.optimizer.zero_grad()


    def train(self, losses=None,
                 optimizer=None,
                 train_dataloader=None,
                 total_iterations=None,
                 val_dataloader = None,
                 lr=None,
                 accumulation_steps=None,
                 scheduler= None,
                 device = "cpu",
                 **kwargs):

        train_dataloader = self.train_dataloader if train_dataloader is None else train_dataloader
        val_dataloader = self.val_dataloader if val_dataloader is None else val_dataloader
        total_iterations = self.total_iterations if total_iterations is None else total_iterations
        accumulation_steps = self.accumulation_steps if accumulation_steps is None else accumulation_steps
        total_iterations = self.total_iterations if total_iterations is None else total_iterations

        self.losses = self.losses if losses is None else losses
        self.lr = self.lr if lr is None else lr
        self.optimizer = self.optimizer if optimizer is None else optimizer
        self.scheduler = self.scheduler if scheduler is None else scheduler

        pbar = tqdm(total=total_iterations, desc='Training', unit='batch', position=0)

        batch_maker = iter(train_dataloader)
        for self.iteration in range(total_iterations):

            # indo para o proximo batch
            try:
              batch = next(batch_maker)
            except StopIteration:
              batch = next(iter(train_dataloader))

            # computa a funcao de perda e os gradientes
            loss = self.compute_loss(batch)

            del batch

            loss = loss / self.accumulation_steps
            loss.backward()

            # atualiza os parametros
            self.gradient_update()

            # if self.val_dataloader:
              # self.validate()

            pbar.set_postfix({
                'Loss': loss.item(),
                # 'lr': self.lr.item()
                })
            self.loss_values.append(loss.item())

            pbar.update(1)


        gc.collect()
        torch.cuda.empty_cache()

        # Adicione sua lógica aqui para calcular métric as ou fazer avaliações durante o treinamento
        # Certifique-se de fechar a barra de progresso após o treinamento
        pbar.close()
        self.plot_loss_curve()



    def plot_loss_curve(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.loss_values, label='Training Loss')
        plt.xlabel('Iteration')
        plt.ylabel('Loss')
        plt.title('Training Loss Curve')
        plt.legend()
        plt.show()



# #############################



import torch
import gc

import torch
import gc
from tqdm import tqdm

def get_batch_labels(model, dataloader, device="cuda"):
    preds = []
    labels = []

    model.eval()
    with torch.no_grad(), tqdm(total=len(dataloader), desc='Processing Batches') as pbar:
        for batch in dataloader:
            # print(batch)
            x = Trainer.process_batch(batch[0], device)
            pred = model(x)
            x = Trainer.process_batch(x, 'cpu')
            pred = Trainer.process_batch(pred, 'cpu')

            preds.append(pred)
            labels.append(batch[1])

            del x

            pbar.update(1)

        gc.collect()
        torch.cuda.empty_cache()

    return preds, labels

# Exemplo de chamada da função
# Substitua 'sua_funcao_que_retorna_as_predicoes' pelo nome da função em seu modelo
# que retorna as predições desejadas.
# preds, labels = get_batch_labels(model_s2, train_dataloader, Trainer)




def configure_optimizer(self, model, lr):
    # Função para configurar o otimizador
    self.optimizer = Adam(model.parameters(), lr=lr)
    self.scheduler = LambdaLR(self.optimizer, lr_lambda=lambda epoch: 0.7 ** (epoch // 20) if 0.7 ** (epoch // 20) > 1e-2 else 0.05)


def simulate_and_plot_lr_schedule(self, total_iterations_sim=None, lr = None):
    dumb_model = nn.Linear(1, 1)
    if lr:
      # Configurando o otimizador usando a função configuradora
      self.configure_optimizer(dumb_model, lr=lr)
    else:
      self.configure_optimizer(dumb_model, lr=self.lr)

    if total_iterations_sim:
      total_iterations_sim = total_iterations_sim
    else:
      total_iterations_sim = self.total_iterations

    lr_values = []
    for self.iteration in range(total_iterations_sim):
      self.gradient_update()
      current_lr = self.optimizer.param_groups[0]['lr']
      lr_values.append(current_lr)

    # default padron
    self.configure_optimizer(self.model, self.lr)


    # Plotando os valores do learning rate
    plt.plot(lr_values)
    plt.xlabel('Iteration')
    plt.ylabel('Learning Rate')
    plt.title('Learning Rate Schedule during Training Simulation')
    plt.show()
    return lr_values






## metrics

In [ ]:
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from scipy.stats import gaussian_kde

import matplotlib as mpl
# mpl.rcParams['hatch.linewidth'] = 0.1  # previous pdf hatch linewidth
mpl.rcParams['hatch.linewidth'] = 5.0  # previous svg hatch linewidth

def plot_density(data, color, label, reverse = False):
    # Sort the data
    data_sorted = np.sort(data)
    print((data_sorted.min(), data_sorted.max()),label)

    ##### estimative of cumulative density
    # Estimate the density values
    # print("data")
    # print(data_sorted)
    kde = gaussian_kde(data_sorted)
    ym = kde(data_sorted)

    # Normalize y-values between 0 and 1
    ym_normalized = (ym - ym.min()) / (ym.max() - ym.min())

    zero = np.array([0])

    # Extend data_sorted and ym_normalized with zeros at the beginning and end
    data_sorted_extended = np.concatenate([data_sorted[:1], data_sorted, data_sorted[-1:]])
    ym_normalized_extended = np.concatenate([zero, ym_normalized, zero])
    ym = np.concatenate([zero, ym, zero])
    #####    #####    #####

    # Plot the density estimates with normalized y-values
    plt.plot(data_sorted_extended, ym_normalized_extended, label=f"Densidade {label}", color=color)
    plt.fill_between(data_sorted_extended, ym_normalized_extended, color=color, alpha=0.3)

    if reverse:
      cum = 1-np.cumsum(ym)/np.sum(ym)
    else:
      cum = np.cumsum(ym)/np.sum(ym)

    plt.plot(data_sorted_extended, cum, label=f"Acumulada {label}", color=color, linestyle='--')
    plt.fill_between(data_sorted_extended, cum, color=color, alpha=0.3,  hatch='*')
    plt.ylabel('Densidade', fontsize=17, fontweight='bold')
    plt.xlabel('Valor', fontsize=17, fontweight='bold')
    plt.yticks(fontsize=14)
    plt.xticks(fontsize=14)

    plt.grid(True)


# Função para calcular WSS com base em um recall desejado
def metrics(probs, labels, recall_target=0.95):
    fpr, tpr, thresholds = roc_curve(labels, probs)

    # Encontrar o índice correspondente ao recall desejado
    idx_target_recall = sum(tpr < recall_target)

    # Obter o limiar correspondente ao recall desejado
    threshold_target_recall = thresholds[idx_target_recall]
    predict_trash = (probs > threshold_target_recall).astype(int)


    # probs[idx_target_recall:]
    # Prever com base no limiar
    # predict_trash = torch.sigmoid(logit).squeeze() >= threshold_target_recall

    # Calcular a matriz de confusão
    CM = confusion_matrix(labels, predict_trash)
    tn, fp, fne, tp = CM.ravel()

    # Calcular as métricas
    P = tp + fne
    N = tn + fp
    recall = tp / P
    wss = (tn + fne) / len(labels) - (1 - recall)
    awss = (tn / N - fne / P)
    f1 = f1_score(predict_trash, predict_trash)


    # print(predict_trash)

    return {
        "wss": round(wss, 4),
        "awss": round(awss, 4),
        "R": round(recall, 4),
        "f1": round(f1, 4),
        "CM": CM,
        'prox.': np.mean((predict_trash == labels)).astype(float)
    }

import torch
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

def plot_metrics(y_scores, y_true, recall_target=0.95, save=None):
    # Calcula a curva ROC
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    # Plota a curva ROC
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='cuvra ROC (area = {:.2f})'.format(roc_auc))
    plt.axhline(y=recall_target, color='red', linestyle='--', label=f'Recall esperado ({recall_target})')

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('Falso Positivo (%)', fontweight='bold', fontsize=14)
    plt.ylabel('Verdadeiro Positivo (%)', fontweight='bold', fontsize=14)
    # plt.title('Receiver Operating Characteristic (ROC) Curve')
    legend = plt.legend(loc='lower right', fontsize=10)

    for text in legend.get_texts():
      text.set_fontweight('bold')

    # Calcula a matriz de confusão e outras métricas
    y_hat = (y_scores > 0.5)
    # y_true = y_true

    CM = confusion_matrix(y_true, y_hat)
    tn, fp, fne, tp = CM.ravel()

    # Calcular as métricas
    P = tp + fne
    N = tn + fp
    recall = tp / P
    wss = (tn + fne) / len(y_true) - (1 - recall)
    awss = (tn / N - fne / P)

    # Assuming y_true and y_pred are your ground truth and predicted labels
    f1 = f1_score(y_true, y_hat)


    # Plot a matriz como cores usando um mapa de calor
    plt.subplot(1, 2, 2)
    m_normalized = np.divide(CM, CM.sum(axis=1, keepdims=True))
    plt.imshow(m_normalized, cmap='viridis', interpolation='nearest')
    plt.colorbar()
    # plt.title('Confusion Matrix as Colors')

    plt.tight_layout()

    if save is not None:
        plt.savefig(f'{save}/plt1.pdf')  # Salva como PNG

    plt.show()

    plt.figure(figsize=(12, 5))
    plot_density(y_scores[y_true == 0], 'blue', '-1', reverse=True)
    plot_density(y_scores[y_true == 1], 'orange', '1')
    plt.axvline(x=0.5, color='red', linestyle='--')
    # Set the axis limits
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    # Add labels and title
    # plt.title('Normalized Probability Density Estimation')

    # Show legend
    legend = plt.legend(fontsize=12, loc='upper center')

    for text in legend.get_texts():
      text.set_fontweight('bold')

    if save is not None:
        plt.savefig(f'{save}/plt2.pdf')  # Salva como PNG

    plt.show()

    return {
     "95": metrics(y_scores, y_true),
     'observed': {
        "wss": round(wss, 4),
        "awss": round(awss, 4),
        "f1": round(f1, 4),
        "R": round(recall, 4),
        "CM": CM,
        "roc_auc":roc_auc,
        'prox.': np.mean(y_hat == y_true).astype(float),
        "fpr":fpr,
        "tpr":tpr,
    }
    }




def plot_rocs(x_list, names):
    plt.figure(figsize=(12, 5))
    for x, name in zip(x_list, names):
      fpr = x.get("fpr")
      tpr = x.get("tpr")
      roc_auc = x.get("roc_auc")
      plt.plot(fpr, tpr, lw=2, label=f'{name} (área = {roc_auc:.2f})')

    plt.axhline(y=0.95, color='red', linestyle='--', label=f'Recall 95%')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('Falso Positivo (%)')
    plt.ylabel('Verdadeiro Positivo (%)')
    # plt.title('Curva ROC')
    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.show()

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

def plot_cosine_similarity_matrix(similarities, vmin=-1.0, vmax=1.0):
    """
    Plot the cosine similarity matrix with a horizontal colorbar.

    Args:
    - similarities (torch.Tensor): Cosine similarity matrix
    - vmin (float): Minimum value for colormap (default: -1.0)
    - vmax (float): Maximum value for colormap (default: 1.0)
    """
    # plt.figure(figsize=(8, 6))
    plt.figure(figsize=(10, 10))

    plt.imshow(similarities.numpy(), interpolation='none',
                vmin=vmin, vmax=vmax)

    # colorbar = plt.colorbar(label='Similaridade de Cosseno')
    # colorbar.ax.tick_params(labelsize=20)  # Set font size for colorbar ticks
    # colorbar.ax.set_ylabel('Similaridade de Cosseno', fontsize=22, weight="bold")  # Set font size and weight for colorbar label

       # plt.title('Matriz de similaridade de cossenos')
    plt.xlabel('Índice', fontsize=20, weight="bold")
    plt.ylabel('Índice', fontsize=20, weight="bold")
    plt.yticks(fontsize=18)
    plt.xticks(fontsize=18)


    plt.show()



In [ ]:

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt






def evaluate_model(model, train_dataloader, test_dataloader, device, dtype=torch.float32):
    emb_test_preds = get_batch_labels(model, test_dataloader, device)  # Assuming Trainer is defined
    emb_train_preds = get_batch_labels(model, train_dataloader, device)

    emb_test_preds = [
        torch.cat(emb_test_preds[0]).to("cpu").squeeze().to(dtype=dtype),
        torch.cat(emb_test_preds[1]).to("cpu").squeeze().to(dtype=dtype)
    ]

    emb_train_preds = [
        torch.cat(emb_train_preds[0]).to("cpu").squeeze().to(dtype=dtype),
        torch.cat(emb_train_preds[1]).to("cpu").squeeze().to(dtype=dtype)
    ]

    gc.collect()
    torch.cuda.empty_cache()

    return emb_test_preds, emb_train_preds


def visualize_tsne(representations, labels,
                   n_components=2,
                   perplexity=30, save=None, alpha=1):
    tsne = TSNE(n_components=n_components, perplexity=perplexity, random_state=42)
    embeddings_2d = tsne.fit_transform(representations)

    plt.figure(figsize=(10, 10))

    for i in labels.unique():
        indices = labels == i
        plt.scatter(embeddings_2d[indices, 0],
                    embeddings_2d[indices, 1], label=f'Classe {int(i)}', alpha=alpha)

    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)

    # plt.title(f't-SNE do Model Embeddings ({n_components} components)')
    legend = plt.legend(fontsize=14)

    for text in legend.get_texts():
      text.set_fontweight('bold')


    if save is not None:
      plt.savefig(f'{save}/tsne.pdf')

    plt.show()

def logistic_regression_probabilities(Train, Test= None, random_state=None):
    """
    Realiza a regressão logística e retorna as probabilidades de pertencer à classe 1.

    Parâmetros:
    - features: Um array-like bidimensional (n, k) contendo as características dos exemplos.
    - categories: Uma lista de categorias (0, 1) para cada exemplo.
    - test_size: A proporção do conjunto de dados a ser utilizado como conjunto de teste.
    - random_state: Semente para controle de reprodutibilidade.

    Retorna:
    - Uma lista de probabilidades para pertencer à classe 1 para cada exemplo.
    """
    # Dividindo o conjunto de dados em treino e teste
    # X_train, X_test, y_train, y_test = train_test_split(features, categories, test_size=test_size, random_state=random_state)
    preds, label = Train
    # Criando e treinando o modelo de regressão logística
    model = LogisticRegression(max_iter=1000)
    model.fit(preds, label)

    # Obtendo as probabilidades no conjunto de teste
    probabilities = model.predict_proba(preds)[:, 1]

    if Test != None:
      return model.predict_proba(Test)[:, 1]

    return probabilities




def eval_pipeline(model, train_dataloader, test_dataloader, device="cuda", perplexity=30, save=None, tsne=False):


    emb_test_preds, emb_train_preds = evaluate_model(model, train_dataloader,
                                                     test_dataloader, device)
    if tsne:
      visualize_tsne(emb_test_preds[0], emb_test_preds[1], perplexity =perplexity, save=save)



    probs = logistic_regression_probabilities(
        Train = emb_train_preds,
        Test = emb_test_preds[0])

    # print(emb_test_preds[1].numpy())
    res = plot_metrics(probs,
                        emb_test_preds[1].numpy(),
                        recall_target=0.95,
                        save=save)
    return res



## Utils

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torch.optim.lr_scheduler import LambdaLR
######

##########################################################################


def main_pipe_train(smodel, submod, df_train,train_dataloader, train_dataloader_eval, test_dataloader_eval, config, save=None):
    torch.cuda.empty_cache()

    # Mod(smodel_mod_t)
    Mod(smodel)
    smodel.util.map(
        name_module = ["query", 'value'] + ["key", 'dense'])


    # activate in the last layers
    Mod(submod)
    submod.util.activate(
        layer_names= ["query", 'value'],
        key = 'set',
        config = {
            'name':'test',
            **config
            })


    smodel.util.trainable_params()
    smodel = smodel.to(smodel.dtype)

    def lr_lambda(epoch):
        return 1 - (epoch / total_iterations)**2

    n_examples = len(df_train)*config["n_pairs"]

    interation = int(n_examples/config["batch_size"])
    total_iterations = config["n_epocs"]*(interation)


    # Create the linear learning rate scheduler
    optimizer = Adam(smodel.parameters(), lr=config["lr"])
    scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)

    #### train
    # smodel
    trainer = Trainer(losses=loss_sim(smodel, tau = config["tau"], device="cuda").to("cuda"),
                      optimizer=optimizer,
                      scheduler=scheduler,
                      accumulation_steps = config["accumulation_steps"],
                      train_dataloader=train_dataloader,
                      total_iterations=total_iterations,
                      device = 'cuda')


    print(f"Quantidade de amostras\n {n_examples}")
    print(f"Quantidade de exemplos\n {len(df_train)}")



    w = smodel.model.encoder.layer[-1].attention.self.value.Mix['Adapt'].loraA

    print(submod)
    print()
    print(w)

    trainer.train()

    print(w)
    smodel.to('cpu')
    torch.cuda.empty_cache()


    from pprint import pprint
    ##########################################################################

    torch.cuda.empty_cache()
    smodel.cuda()
    res = eval_pipeline(smodel,
                        train_dataloader=train_dataloader_eval,
                        test_dataloader=test_dataloader_eval,
                  device="cuda", save=save)
    smodel.cpu()


    pprint(res)


    torch.cuda.empty_cache()
    return res, trainer


def eval_model_emb(smodel, lod):
  smodel.cuda()
  preds= []
  labels= []
  for i,lo in enumerate(lod):
    emb_test, emb_train = evaluate_model(smodel,*lo, "cuda")
    # print(emb_test[1])
    i = i+1
    emb_test[1][emb_test[1]==1] = i
    emb_test[1][emb_test[1]==0] = -i
    # print(emb_test[1])

    preds.append(emb_test[0])
    labels.append(emb_test[1])

  smodel.cpu()
  torch.cuda.empty_cache()
  if len(lod)>1:
    preds = torch.concat(preds)
    labels = torch.concat(labels)
  else:
    preds, labels = preds[0], labels[0]
  return preds, labels


def export_adapt_head(smodel, submod):
  w = submod.util.activate(
    layer_names= ["query", 'value'],
    key = 'export',
    Modular=Lora)
  if smodel.head is not None:
    head_w = copy.deepcopy(smodel.head.state_dict())
  else:
    head_w  = None
  return w, head_w

def load_adapt_head(smodel,submod, adapt, head_w):
  for w in adapt:
    submod.get_submodule(w[0]).load_state_dict(w[1],  strict=False)

  if smodel.head is not None:
    smodel.head.load_state_dict(head_w)

# load_adapt_head(smodel, w1, head1_w)



## 🟡 Pipe

### Models ref (teacher) 🎩

In [ ]:
from transformers import AutoTokenizer, AutoModel

from transformers import  BitsAndBytesConfig


model_name = 'allenai/specter2_base'

# model_name = 'sentence-transformers/paraphrase-MiniLM-L3-v2'
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'
# model_name = 'allenai/scibert_scivocab_uncased'

model_ref_t = AutoModel.from_pretrained(model_name)
tokenizer_t_ref = AutoTokenizer.from_pretrained(model_name)


# model_ref_t.encoder.layer[0].attention.output.dense.weight


import copy

max_seq_length = 512

tokenize_t = lambda train_text: tokenizer_t_ref(train_text,
    padding="max_length",
    max_length=max_seq_length,
    truncation=True,
    return_tensors="pt")


tokenize_batch_t = lambda train_text: tokenizer_t_ref.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

# base = "/content/gdrive/MyDrive/Projetos/TCC"
# base_save = f"{base}/export/diagnostic/{model_name}"
# create_repo_if_not_exist(base_save+"/")
# file_path = f"/content/content/slr_data_tcc/{base_dataset}"


##### modular
Mod = ModularUtils(Modular=ModMix)
Mod.freeze(module=model_ref_t)

smodel_mod_t = SentenceEmbedding(
    model_ref_t,
    normalize=False,
    with_head = False
    )


smodel_mod_t, model_name

### big model

temperature of softmax

the magnitude of positive gradient is equal to the sum of negative gradients. The temperature controls the distribution of negative gradients.

Smaller temperature tends to concentrate more on the nearest neighbours of the anchor point, which plays a role in controlling the hardness-aware sensitivity


This can be explained as follows: when the temperature is small, the contrastive loss tends to separate the positive samples
close to the anchor sample, which makes the local distribution be sparse. With all samples are trained, the embedding
space tends to make the neighbour of each point be sparse, and the distribution tends to be more uniform

We could see that the tolerance is positively related to the temperature τ . However, the tolerance can not directly reflect the feature quality. For
example, when all the samples reside in a single point of the hypersphere, then the tolerance is maximized, while the
feature quality is bad. The tolerance reflects the local density of semantically related samples. An ideal model should
be both locally clustered and globally uniform.

#### data

In [ ]:

import numpy as np
import pandas as pd
import torch

# Set seed for NumPy

# seed = 21442

np.random.seed(seed)

# Set seed for PyTorch
torch.manual_seed(seed)

# Set seed for CUDA if available
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)


config = {
    "total_batch": 16,
    "accumulation_steps": 1,
    "batch_size":int(16/1),
    "batch_size_eval":16,
###
    "n_epocs": 2,
    "n_pairs" : 40,
    "lr" : 9e-3,
###
    "alpha":4*(0.1),
    "r":4,
    # "tau" : 0.05,
    "tau" : 0.1,
###
    "la" : 3,
    "rest" : 12 - 3,
           }

######
##########################################################################
##### modular

base_dataset1 = "cohen/ADHD.csv"
file_path = f"data/content/slr_data_tcc/{base_dataset1}"

df_train1, exs_test1 = data_import(file_path, n=8, samples_n=300, samples_p=100)
train_dataloader1, test_dataloader_eval1,  train_dataloader_eval1 = loaders(df_train1, exs_test1, config)


# base_dataset = "synergy/Smid_2020.csv"
base_dataset2 = "SWIFT systematic review data/BPA.csv"
file_path = f"data/content/slr_data_tcc/{base_dataset2}"

df_train2, exs_test2 = data_import(file_path, n=8, samples_n=300, samples_p=100)
train_dataloader2, test_dataloader_eval2,  train_dataloader_eval2 = loaders(df_train2, exs_test2, config)


base_dataset3 = "SWIFT systematic review data/Fluoride.csv"
file_path = f"data/content/slr_data_tcc/{base_dataset3}"

df_train3, exs_test3 = data_import(file_path, n=8, samples_n=300, samples_p=100)
train_dataloader3, test_dataloader_eval3,  train_dataloader_eval3 = loaders(df_train3, exs_test3, config)


base_dataset4 = "SWIFT systematic review data/PFOS-PFOA.csv"
file_path = f"data/content/slr_data_tcc/{base_dataset4}"

df_train4, exs_test4 = data_import(file_path, n=8, samples_n=300, samples_p=100)
train_dataloader4, test_dataloader_eval4,  train_dataloader_eval4 = loaders(df_train4, exs_test4, config)


# base_dataset5 = "cohen/Opiods.csv"
# file_path = f"data/content/slr_data_tcc/{base_dataset5}"


# df_train5, exs_test5 = data_import(file_path, n=8, samples_n=300, samples_p=100)
# train_dataloader5, test_dataloader_eval5,  train_dataloader_eval5 = loaders(df_train5, exs_test5, config)


base_dataset6 = "SWIFT systematic review data/Transgenerational.csv"
file_path = f"data/content/slr_data_tcc/{base_dataset6}"

df_train6, exs_test6 = data_import(file_path, n=8, samples_n=300, samples_p=100)
train_dataloader6, test_dataloader_eval6,  train_dataloader_eval6 = loaders(df_train6, exs_test6, config)




lod1 = (train_dataloader_eval1, test_dataloader_eval1)
lod_train1= (df_train1, train_dataloader1, *lod1)

lod2 = (train_dataloader_eval2, test_dataloader_eval2)
lod_train2= (df_train2, train_dataloader2, *lod2)

lod3 = (train_dataloader_eval3, test_dataloader_eval3)
lod_train3= (df_train3, train_dataloader3, *lod3)

lod4 = (train_dataloader_eval4, test_dataloader_eval4)
lod_train4= (df_train4, train_dataloader4, *lod4)

# lod5 = (train_dataloader_eval5, test_dataloader_eval5)
# lod_train5= (df_train5, train_dataloader5, *lod5)

lod6 = (train_dataloader_eval6, test_dataloader_eval6)
lod_train6= (df_train6, train_dataloader6, *lod6)

#### No train

In [ ]:
from transformers import AutoTokenizer, AutoModel



model_name = 'allenai/specter2_base'
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'



model_ref_t = AutoModel.from_pretrained(model_name)
tokenizer_t_ref = AutoTokenizer.from_pretrained(model_name)


config = {
    "total_batch": 16,
    "accumulation_steps": 1,
    "batch_size":int(16/1),
    "batch_size_eval":16,
###
    "n_epocs": 2,
    "n_pairs" : 40,
    "lr" : 9e-3,
###
    "alpha":4*(0.1),
    "r":4,
    # "tau" : 0.05,
    "tau" : 0.2,
###
    "la" : 3,
    "rest" : 12 - 3,
           }



import copy

max_seq_length = 512

tokenize_t = lambda train_text: tokenizer_t_ref(train_text,
    padding="max_length",
    max_length=max_seq_length,
    truncation=True,
    return_tensors="pt")


tokenize_batch_t = lambda train_text: tokenizer_t_ref.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

# base = "/content/gdrive/MyDrive/Projetos/TCC"
# base_save = f"{base}/export/diagnostic/{model_name}"
# create_repo_if_not_exist(base_save+"/")
# file_path = f"/content/content/slr_data_tcc/{base_dataset}"


##### modular
Mod = ModularUtils(Modular=ModMix)
Mod.freeze(module=model_ref_t)

smodel = SentenceEmbedding(
    copy.deepcopy(model_ref_t),
    normalize=False,
    with_head = False
    )

smodel = smodel.half()



#### speed

In [ ]:
def data_import(file_path, n=8, samples_n=200, samples_p=1):
    print("####"*5)
    print()
    print(os.path.basename(file_path))
    df = preprocess_csv_file(file_path)


    tk = lambda x: len(tokenizer_t_ref.encode(x))
    tk_len = df.text.map(tk)
    tk_len.hist()


    df_rest = df[tk_len > 20]
    df_train = df_rest.groupby('label').sample(n)

    print("data count:")
    print(df['label'].value_counts())

    print(df_train.iloc[0])

    df_test = df[~df.index.isin(df_train.index)]

    if len(df_test[df_test['label'] == 1]) < samples_p:
        df_test_one = df_test[df_test['label'] == 1].copy()
    else:
        df_test_one = df_test[df_test['label'] == 1].sample(n=samples_p, replace=False)


    if len(df_test[df_test['label'] == 0]) < samples_n:
        df_test_zero = df_test[df_test['label'] == 0].copy()
    else:
        df_test_zero = df_test[df_test['label'] == 0].sample(n=samples_n, replace=False)

    exs_test = pd.concat([df_test_one,df_test_zero])

    print("Eval count:")
    print(exs_test['label'].value_counts())
    return df_train, exs_test


def loaders(df_train, exs_test, config):
    sim_data = SIMDataset(df_train, tokenize_batch_t, config["n_pairs"],
                          )
    train_dataloader = DataLoader(sim_data,shuffle=True,
                                  batch_size=config["batch_size"])

    test_dataloader_eval = DataLoader(shuffle=False, batch_size=config["batch_size_eval"],
                dataset=SIMDataset(
                    exs_test, tokenize_batch_t, 0,
                          )
                )

    train_dataloader_eval = DataLoader(shuffle=False, batch_size=config["batch_size"],
                dataset=SIMDataset(
                    df_train, tokenize_batch_t, 0,
                    # emb= emb, device="cuda", pttn="* d a"
                          )
                )

    return train_dataloader, test_dataloader_eval,  train_dataloader_eval



# path= "data/content/slr_data_tcc/cohen/Opiods.csv"
path= "./content/slr_data_tcc/cohen/Opiods.csv"
df = preprocess_csv_file(path)

tk = lambda x: len(tokenizer_t_ref.encode(x))
tk_len = df.text.map(tk)

ex = df[tk_len>=512]

ex = ex.iloc[:1,:]
all = []
for _ in range(200):
    all.append(ex)
ex = pd.concat(all)

eval_speed_dataloader = DataLoader(SIMDataset(ex, tokenize_batch_t, 0),
                              shuffle=False,
                              batch_size=200)


smodel.to(torch.bfloat16)

import time

start_time = time.time()


for x in eval_speed_dataloader:
  smodel(x[0])
end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")








In [ ]:
!pip install onnx
!pip install onnxruntime

In [ ]:
import torch
from transformers import BertModel, BertTokenizer

input_text = "Your input text with 512 tokens."

smodel.to(torch.float32)


# Tokenize input with truncation to max_length
inputs = tokenizer_t_ref(input_text, truncation=True, return_tensors="pt",  padding='max_length', max_length=512)


# Export the model to ONNX format
torch.onnx.export(smodel.model, (inputs["input_ids"], inputs["attention_mask"]), "bert_model.onnx",
                  input_names = ['input_ids', "attention_mask"],
                                    dynamic_axes={"input_ids": {0: "batch_size"}, "attention_mask": {0: "batch_size"}})


# Quantize the ONNX model to 8-bit integers
from onnxruntime.quantization import quantize_dynamic

quantize_dynamic("bert_model.onnx", "quantized_bert_model.onnx")


In [ ]:
import onnxruntime
import numpy as np
import time

# Load the quantized BERT model
ort_session = onnxruntime.InferenceSession("quantized_bert_model.onnx")

# Generate fake input data with 512 tokens
fake_data = " ".join(["token" + str(i) for i in range(512)])

# Tokenize input data
input_single = tokenizer_t_ref(fake_data, return_tensors="np",max_length=512)

# Repeat the single example to create a batch of 32 examples
input_ids = np.repeat(input_single["input_ids"], 32, axis=0)
attention_mask = np.repeat(input_single["attention_mask"], 32, axis=0)

input_ids = input_ids.reshape(32, 512)
attention_mask = attention_mask.reshape(32, 512)

# Run inference and measure time
start_time = time.time()
outputs = ort_session.run(None, {"input_ids": input_ids.astype(np.int64), "attention_mask": attention_mask.astype(np.int64)})
end_time = time.time()

# Calculate inference time
inference_time = end_time - start_time
print("Total inference time for a batch of 32 examples:", inference_time, "seconds")


In [ ]:

# Export the model to ONNX format
torch.onnx.export(smodel.model, (inputs["input_ids"], inputs["attention_mask"]), "bert_model.onnx", verbose=True)

# Load the ONNX model
import onnx

onnx_model = onnx.load("bert_model.onnx")

# Quantize the ONNX model to 8-bit integers
from onnxruntime.quantization import quantize_dynamic

quantized_model = quantize_dynamic(onnx_model, per_channel=True)

# Save the quantized model
onnx.save(quantized_model, "quantized_bert_model.onnx")

# Now you can use the quantized_bert_model.onnx for inference on 8-bit CPUs


#### normal

In [ ]:
import glob
import os
from pprint import pprint

# base_path = 'data/content/slr_data_tcc'
base_path = './content/slr_data_tcc'

# Paths and names of the domains datasets
# path = 'SLR_data'
paths_cohen = glob.glob(f"{base_path}/cohen/*.csv", recursive=True)
# pprint(paths_cohen)

paths_swift = glob.glob(f"{base_path}/SWIFT systematic review data/*.csv", recursive=True)
# pprint(paths_swift)


# names = [os.path.basename(p) for p in paths]

import random
paths = [*paths_swift, *paths_cohen]

paths_n = random.sample(paths, k=5)

pprint(paths_n)

In [ ]:
# df_train, exs_test = data_import(p, n=8, samples_n=300, samples_p=100)



def visualize_tsne(representations, labels,
                   n_components=2,
                   perplexity=30, save=None, alpha=1):
    tsne = TSNE(n_components=n_components, perplexity=perplexity, random_state=42)
    embeddings_2d = tsne.fit_transform(representations)

    plt.figure(figsize=(10, 10))

    for i in labels.unique():
        indices = labels == i
        plt.scatter(embeddings_2d[indices, 0],
                    embeddings_2d[indices, 1], label=f'Classe {int(i)}', alpha=alpha)

    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)

    # plt.title(f't-SNE do Model Embeddings ({n_components} components)')
    legend = plt.legend(fontsize=14)

    for text in legend.get_texts():
      text.set_fontweight('bold')


    if save is not None:
      plt.savefig(f'{save}/tsne.pdf')

    plt.show()

In [ ]:
lods=[]

for p in paths_n:

  # p, l = eval_model_emb(smodel_test, [lod1,lod2])

  df_train, exs_test = data_import(p, n=8, samples_n=300, samples_p=100)
  train_dataloader, test_dataloader_eval,  train_dataloader_eval = loaders(df_train, exs_test, config)

  lod = (train_dataloader_eval, test_dataloader_eval)

  lods.append(lod)


p, l = eval_model_emb(smodel, lods)

# save = f"plots/results/emd_wo_train"
# create_repo_if_not_exist(save)
visualize_tsne(p, l, perplexity =7)






# smodel_test = SentenceEmbedding(
#     copy.deepcopy(model_ref_t),
#     normalize=False,
#     with_head = False,
#     )


# Mod(smodel_test)
# smodel_test.util.map(
#     name_module = ["query", 'value'] + ["key", 'dense'])


# # activate in the last layers
# submod_test = smodel_test.model.encoder.layer[-config["la"]:]
# Mod(submod_test)
# submod_test.util.activate(
#     layer_names= ["query", 'value'],
#     key = 'set',-
#     config = {
#         'name':'test',
#         "alpha":16*(1),
#         "r":16,
#         })
# load_adapt_head(smodel_test, submod_test, *w_5)




In [ ]:


# path= "data/content/slr_data_tcc/cohen/Opiods.csv"
path= "./content/slr_data_tcc/cohen/NSAIDS.csv"

df_train, exs_test = data_import(path, n=8, samples_n=300, samples_p=100)
train_dataloader, test_dataloader_eval,  train_dataloader_eval = loaders(df_train, exs_test, config)

lod_eval = (train_dataloader_eval, test_dataloader_eval)


In [ ]:

def visualize_tsne(representations, labels,
                   n_components=2,
                   perplexity=30, save=None, alpha=1):
    tsne = TSNE(n_components=n_components, perplexity=perplexity, random_state=42)
    embeddings_2d = tsne.fit_transform(representations)

    plt.figure(figsize=(10, 10))

    for i in labels.unique():
        indices = labels == i
        plt.scatter(embeddings_2d[indices, 0],
                    embeddings_2d[indices, 1],
                    label=f'Classe {int(i)}', alpha=alpha, s = 100)

    plt.yticks(fontsize=18)
    plt.xticks(fontsize=18)

    # plt.title(f't-SNE do Model Embeddings ({n_components} components)')
    legend = plt.legend(fontsize=18)

    for text in legend.get_texts():
      text.set_fontweight('bold')


    if save is not None:
      plt.savefig(f'{save}/tsne.pdf')

    plt.show()



smodel_test = copy.deepcopy(smodel)
smodel_test = smodel_test.to(torch.float32)
smodel_test = smodel_test.to(torch.bfloat16)
submod = smodel_test.model.encoder.layer[-config["la"]:]

# submod =submod.half()

p0, l0 = eval_model_emb(smodel, [lod_eval])
# visualize_tsne(p, l, perplexity =20, save=None)

save = f"plots/results/emd_example/data_wo/"
create_repo_if_not_exist(save)




lod_train= (df_train, train_dataloader, train_dataloader_eval, test_dataloader_eval)
_ = main_pipe_train(smodel_test, submod, *lod_train ,config)


p1, l1 = eval_model_emb(smodel_test, [lod_eval])

save = f"plots/results/emd_example/data_w/"
create_repo_if_not_exist(save)

visualize_tsne(p0, l0, perplexity =15, save=save)
visualize_tsne(p1, l1, perplexity =15, save=save)




In [ ]:
# 12*((64**2+64**2+64**2+64**2)*12 + 768*3072 + 3070*768)/10**8

In [ ]:
model = smodel_test
submod_test = model.model.encoder.layer[-config["la"]:]
lod = lod_eval


# w = merge([
#                    w_1[0],
#                    w_2[0],
#                   #  w_3[0],
#                   #  w_4[0],
#                   #  w_6[0],
#     ],
#                   [
#                    w_1[1],
#                    w_2[1],
#                   #  w_3[1],
#                   #  w_4[1],
#                   #  w_6[1]
#                       ],
#                   [
#                    0.7,
#                    0.7,
#                   #  0.2,
#                   # 0.2,
#                   # 0.2
#                   #  1
#                   ],
#                   r=32, p=0.5
#                   )

# load_adapt_head(model, submod_test, *w)


from pprint import pprint
_=model.cuda()
res = eval_pipeline(model,*lod,
                    device="cuda", save=None)
pprint(res)
_=model.cpu()



model.eval()
model.cuda()
emb_test, emb_train = evaluate_model(model ,*lod, "cuda")
model.cpu()


import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
ids_pos = sum(emb_test[1]==1)
print(f"fisst positive samples: 0:{ids_pos}")

# Example usage:
# Assuming emb_test[0] contains the batch of vectors
batch_vectors_normalized = F.normalize(emb_test[0], p=2, dim=1)
similarities = torch.matmul(batch_vectors_normalized, batch_vectors_normalized.t())
plot_cosine_similarity_matrix(similarities[:,:], -1, 1)


In [ ]:
model = smodel
submod_test = model.model.encoder.layer[-config["la"]:]
lod = lod_eval


# w = merge([
#                    w_1[0],
#                    w_2[0],
#                   #  w_3[0],
#                   #  w_4[0],
#                   #  w_6[0],
#     ],
#                   [
#                    w_1[1],
#                    w_2[1],
#                   #  w_3[1],
#                   #  w_4[1],
#                   #  w_6[1]
#                       ],
#                   [
#                    0.7,
#                    0.7,
#                   #  0.2,
#                   # 0.2,
#                   # 0.2
#                   #  1
#                   ],
#                   r=32, p=0.5
#                   )

# load_adapt_head(model, submod_test, *w)


from pprint import pprint
_=model.cuda()
res = eval_pipeline(model,*lod,
                    device="cuda", save=None)
pprint(res)
_=model.cpu()



model.eval()
model.cuda()
emb_test, emb_train = evaluate_model(model ,*lod, "cuda")
model.cpu()


import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
ids_pos = sum(emb_test[1]==1)
print(f"fisst positive samples: 0:{ids_pos}")

# Example usage:
# Assuming emb_test[0] contains the batch of vectors
batch_vectors_normalized = F.normalize(emb_test[0], p=2, dim=1)
similarities = torch.matmul(batch_vectors_normalized,
                            batch_vectors_normalized.t())
plot_cosine_similarity_matrix(similarities[:,:], -1, 1)


#### choose model

In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import  BitsAndBytesConfig


model_names = [
    'allenai/specter2_base',
    'sentence-transformers/paraphrase-MiniLM-L3-v2',
    'sentence-transformers/all-MiniLM-L12-v2',
    'allenai/scibert_scivocab_uncased'
    ]


for model_name in model_names:
  model_ref_t = AutoModel.from_pretrained(model_name)
  tokenizer_t_ref = AutoTokenizer.from_pretrained(model_name)


  # model_ref_t.encoder.layer[0].attention.output.dense.weight


  import copy

  max_seq_length = 512

  tokenize_t = lambda train_text: tokenizer_t_ref(train_text,
      padding="max_length",
      max_length=max_seq_length,
      truncation=True,
      return_tensors="pt")


  tokenize_batch_t = lambda train_text: tokenizer_t_ref.batch_encode_plus(
      train_text,
      add_special_tokens=True,
      padding='max_length',
      max_length=max_seq_length,
      truncation=True,
      return_tensors='pt',
      return_attention_mask=True
  )

  # base = "/content/gdrive/MyDrive/Projetos/TCC"
  # base_save = f"{base}/export/diagnostic/{model_name}"
  # create_repo_if_not_exist(base_save+"/")
  # file_path = f"/content/content/slr_data_tcc/{base_dataset}"


  ##### modular
  Mod = ModularUtils(Modular=ModMix)
  Mod.freeze(module=model_ref_t)

  smodel_mod_t = SentenceEmbedding(
      model_ref_t,
      normalize=False,
      with_head = True
      )


  smodel_mod_t, model_name

  ###### model
  smodel = copy.deepcopy(smodel_mod_t)
  head = copy.deepcopy(smodel.head)
  submod = smodel.model.encoder.layer[-config["la"]:]


  base_save = "metrics"
  save = f"{base_save}/{model_name}/"
  create_repo_if_not_exist(save)


  lod= (df_train5, train_dataloader5, train_dataloader_eval5, test_dataloader_eval5)
  res = main_pipe_train(smodel, submod, *lod ,config, save)
  # w_4 = export_adapt_head(smodel, submod)
  export_json(res[0], f"{save}Metrics.json")

In [ ]:
import glob
import os
from pprint import pprint

base_path = 'metrics/**/Metrics.json'

# Paths and names of the domains datasets
# path = 'SLR_data'
paths_metrics = glob.glob(base_path, recursive=True)
pprint(paths_metrics)

metrics_list=[]
names=[]
for p in paths_metrics:
  metric = read_json(p)
  name = p.split("/")[-2]
  metrics_list.append(metric["observed"])
  names.append(name)

def plot_rocs(x_list, names):
    plt.figure(figsize=(7, 5))
    for x, name in zip(x_list, names):
      fpr = x.get("fpr")
      tpr = x.get("tpr")
      roc_auc = x.get("roc_auc")
      plt.plot(fpr, tpr, lw=2, label=f'{name} (roc_auc = {roc_auc:.2f})')

    plt.axhline(y=0.95, color='red', linestyle='--', label=f'Recall 95%')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

    plt.xlabel('Falso Positivo (%)', fontsize=14)
    plt.ylabel('Verdadeiro Positivo (%)', fontsize=14)
    plt.yticks(fontsize=12)
    plt.xticks(fontsize=12)

    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.savefig('model_choose.pdf')
    plt.show()


plot_rocs(metrics_list,
          names)



#### 16bit

In [ ]:
from transformers import AutoTokenizer, AutoModel

from transformers import  BitsAndBytesConfig


model_name = 'allenai/specter2_base'
# model_name = 'sentence-transformers/paraphrase-MiniLM-L3-v2'
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'
# model_name = 'allenai/scibert_scivocab_uncased'



config = {
    "total_batch": 16,
    "accumulation_steps": 1,
    "batch_size":int(16/1),
    "batch_size_eval":16,
###
    "n_epocs": 2,
    "n_pairs" : 40,
    "lr" : 9e-3,
###
    "alpha":4*(0.1),
    "r":4,
    "tau" : 0.1,
###
    "la" : 3,
    "rest" : 12 - 3,
           }

model_ref_t = AutoModel.from_pretrained(model_name)
tokenizer_t_ref = AutoTokenizer.from_pretrained(model_name)


import copy

max_seq_length = 512

tokenize_t = lambda train_text: tokenizer_t_ref(train_text,
    padding="max_length",
    max_length=max_seq_length,
    truncation=True,
    return_tensors="pt")


tokenize_batch_t = lambda train_text: tokenizer_t_ref.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

##### modular
Mod = ModularUtils(Modular=ModMix)
Mod.freeze(module=model_ref_t)


smodel = SentenceEmbedding(
    model_ref_t,
    normalize=False,
    with_head = False
    )


smodel, model_name

In [ ]:
# config["lr"]


# base_save = "metrics"
# # base_dataset1
# save = f"{base_save}/{model_name}/"
# create_repo_if_not_exist(save)
# export_json(res, f"{save}Metrics.json")


###### model
smodel_test = copy.deepcopy(smodel)
smodel_test=smodel_test.to(torch.bfloat16)
submod = smodel_test.encoder.layer[-config["la"]:]

_ = main_pipe_train(smodel_test, submod, *lod_train1 ,config)
w_1 = export_adapt_head(smodel_test, submod)

##### model
smodel_test = copy.deepcopy(smodel)
smodel_test=smodel_test.to(torch.bfloat16)
submod = smodel_test.encoder.layer[-config["la"]:]

_ = main_pipe_train(smodel_test, submod, *lod_train2 ,config)
w_2 = export_adapt_head(smodel_test, submod)


###### model
smodel_test = copy.deepcopy(smodel)
smodel_test=smodel_test.to(torch.bfloat16)
submod = smodel_test.encoder.layer[-config["la"]:]

_ = main_pipe_train(smodel_test, submod, *lod_train3 ,config)
w_3 = export_adapt_head(smodel_test, submod)


###### model
smodel_test = copy.deepcopy(smodel)
smodel_test=smodel_test.to(torch.bfloat16)
submod = smodel_test.encoder.layer[-config["la"]:]

_ = main_pipe_train(smodel_test, submod, *lod_train4 ,config)
w_4 = export_adapt_head(smodel_test, submod)


# ###### model
# smodel_test = copy.deepcopy(smodel)
# smodel_test=smodel_test.to(torch.bfloat16)
# submod = smodel_test.encoder.layer[-config["la"]:]

# _ = main_pipe_train(smodel_test, submod, *lod_train5 ,config)
# w_5 = export_adapt_head(smodel_test, submod)


###### model
smodel_test = copy.deepcopy(smodel)
smodel_test=smodel_test.to(torch.bfloat16)
submod = smodel_test.encoder.layer[-config["la"]:]

_ = main_pipe_train(smodel_test, submod, *lod_train6 ,config)
w_6 = export_adapt_head(smodel_test, submod)


In [ ]:
import torch

def zero_out_smallest_percentage(tensor, percentage):
    """
    Zero out a certain percentage of smallest values in a PyTorch tensor.

    Args:
    - tensor: input PyTorch tensor
    - percentage: percentage of smallest values to zero out

    Returns:
    - masked_tensor: tensor with specified percentage of smallest values zeroed out
    """
    # Calculate the number of elements to zero out
    num_zeros = int(tensor.numel() * percentage)

    # Find the indices of the smallest values
    _, indices = torch.topk(tensor.abs().reshape(-1), num_zeros, largest=False)

    # Create a mask of zeros and ones with the same shape as the tensor
    mask = torch.ones_like(tensor)

    # Set selected indices to 0
    mask.reshape(-1)[indices] = 0

    # Apply the mask to zero out selected values
    masked_tensor = tensor * mask

    return masked_tensor

# Example usage:


In [ ]:
from einop import einop
def merge(ws, heads, weights=[], r=8, p=0):
  w = copy.deepcopy(ws[0])
  head = copy.deepcopy(heads[0])

  if len(weights)==0:
    weights=[1/len(ws)]*len(ws)

  for i in range(len(w)):
    a = torch.stack([w_temp[i][1]["loraA"]* weights[j] for j, w_temp in enumerate(ws)])
    b = torch.stack([w_temp[i][1]["loraB"]* weights[j] for j, w_temp in enumerate(ws)])

    # a = einop(a, "b i r -> i (b r)")
    # b = einop(b, "b r o ->(b r) o")


    dtype= a.dtype
    a= a.to(torch.float32)
    b= b.to(torch.float32)
    # a = torch.quantile(a, p, dim=0)
    # b = torch.quantile(b, p, dim=0)
    # a[torch.abs(a)<torch.quantile(torch.abs(a), 0.2)] = 0
    # b[torch.abs(b)<torch.quantile(torch.abs(b), 0.2)] = 0


    ww = a@b
    ww = torch.quantile(ww, p, dim=0)
    # ww = zero_out_smallest_percentage(ww, p)
    ww[torch.abs(ww)<torch.quantile(torch.abs(ww), 0.5)] = 0


    U,S,V = torch.svd(ww)
    a =U[:, :r]
    b=  torch.diag(S[:r]) @ V[:, :r].T


    a= a.to(dtype)
    b= b.to(dtype)

    w[i][1]["loraA"] = a
    w[i][1]["loraB"] = b

  return w, head


# head_w_opi = dict(head_w_opi)

r=8
w = merge([
                   w_1[0],
                   w_2[0],
                   w_3[0],
                   w_4[0],
                  #  w_5[0],
                   w_6[0],
    ],
                  [
                   w_1[1],
                   w_2[1],
                   w_3[1],
                   w_4[1],
                  #  w_5[1],
                   w_6[1]
                      ],
                  [
                   1,
                   1,
                   1,
                   1,
                   1,
                   1,
                  ],
                  r=r, p=0.7
                  )





smodel_test_emb = SentenceEmbedding(
    copy.deepcopy(model_ref_t),
    with_head = False,
    normalize=False,
    )



Mod(smodel_test_emb)
smodel_test_emb.util.map(
    name_module = ["query", 'value'] + ["key", 'dense'])


# activate in the last layers
submod_test = smodel_test_emb.model.encoder.layer[-config["la"]:]
Mod(submod_test)
submod_test.util.activate(
    layer_names= ["query", 'value'],
    key = 'set',
    config = {
        'name':'test',
        "r":r,
        "alpha":r*(0.1)
        })


load_adapt_head(smodel_test_emb, submod_test, *w)

########################## eval

smodel=smodel.to(torch.float16)
smodel_test_emb=smodel_test_emb.to(torch.float16)
p0, l0 = eval_model_emb(smodel, [lod1,lod2,lod3,lod4,lod6])
p1, l1 = eval_model_emb(smodel_test_emb, [lod1,lod2,lod3,lod4,lod6])
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt



def visualize_tsne(representations, labels,
                   n_components=2,
                   perplexity=30, save=None, alpha=1):
    tsne = TSNE(n_components=n_components, perplexity=perplexity, random_state=42)
    embeddings_2d = tsne.fit_transform(representations)

    plt.figure(figsize=(10, 10))

    for i in [-1,1,-2,2,-3,3,-4,4,-5,5]:
        indices = labels == i
        plt.scatter(embeddings_2d[indices, 0],
                    embeddings_2d[indices, 1],
                    label=f'Classe {int(i)}', alpha=alpha, s=100)

    plt.yticks(fontsize=18)
    plt.xticks(fontsize=18)
    # plt.xlim(-50, 50)
    # plt.ylim(-50, 50)

    # Place legend outside of plot
    # legend = plt.legend(bbox_to_anchor=(0.5, 1.2), loc='upper center', fontsize=24, ncol=5)

    # for text in legend.get_texts():
    #   text.set_fontweight('bold')

    if save is not None:
      plt.savefig(f'{save}/tsne.pdf')

    plt.show()



visualize_tsne(p0, l0, perplexity =40, save=None, alpha=0.7)
visualize_tsne(p1, l1, perplexity =40, save=None, alpha=0.7)

In [ ]:
smodel_test_emb = smodel_test_emb.to(torch.float16)
model = smodel_test_emb
submod_test = model.model.encoder.layer[-config["la"]:]
lod = lod1


load_adapt_head(model, submod_test, *w)


# from pprint import pprint
# _=model.cuda()
# res = eval_pipeline(model,*lod,
#                     device="cuda", save=None)
# pprint(res)
# _=model.cpu()



model.eval()
model.cuda()
emb_test, emb_train = evaluate_model(model ,*lod, "cuda")
model.cpu()


import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
ids_pos = sum(emb_test[1]==1)
print(f"fisst positive samples: 0:{ids_pos}")

# Example usage:
# Assuming emb_test[0] contains the batch of vectors
batch_vectors_normalized = F.normalize(emb_test[0], p=2, dim=1)
similarities = torch.matmul(batch_vectors_normalized, batch_vectors_normalized.t())
plot_cosine_similarity_matrix(similarities[:,:], -1, 1)


In [ ]:
smodel_test_emb = SentenceEmbedding(
    copy.deepcopy(model_ref_t),
    with_head = False,
    normalize=False,
    )



Mod(smodel_test_emb)
smodel_test_emb.util.map(
    name_module = ["query", 'value'] + ["key", 'dense'])


# activate in the last layers
submod_test = smodel_test_emb.model.encoder.layer[-config["la"]:]
Mod(submod_test)
submod_test.util.activate(
    layer_names= ["query", 'value'],
    key = 'set',
    config = {
        'name':'test',
        "r":4,
        "alpha":4*(0.1)
        })



lod = lod1
smodel_test_emb = smodel_test_emb.to(torch.float16)
model = smodel_test_emb


load_adapt_head(model, submod_test, *w_1)


# from pprint import pprint
# _=model.cuda()
# res = eval_pipeline(model,*lod,
#                     device="cuda", save=None)
# pprint(res)
# _=model.cpu()



model.eval()
model.cuda()
emb_test, emb_train = evaluate_model(model ,*lod, "cuda")
model.cpu()


import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
ids_pos = sum(emb_test[1]==1)
print(f"fisst positive samples: 0:{ids_pos}")

# Example usage:
# Assuming emb_test[0] contains the batch of vectors
batch_vectors_normalized = F.normalize(emb_test[0], p=2, dim=1)
similarities = torch.matmul(batch_vectors_normalized, batch_vectors_normalized.t())
plot_cosine_similarity_matrix(similarities[:,:])


In [ ]:
from pprint import pprint
smodel_test_emb = SentenceEmbedding(
    copy.deepcopy(model_ref_t),
    with_head = False,
    normalize=False,
    )

smodel_test_emb = smodel_test_emb.to(torch.float16)


model = smodel_test_emb


res_cru = []

for lo, wo in zip([lod1, lod2, lod3, lod4, lod6],
                  [w_1, w_2, w_3, w_4, w_6]):
  _=model.cuda()
  res = eval_pipeline(model,*lo,
                      device="cuda", save=None)
  pprint(res)
  _=model.cpu()
  res_cru.append(res)




###########################


smodel_test_emb = SentenceEmbedding(
    copy.deepcopy(model_ref_t),
    with_head = False,
    normalize=False,
    )

smodel_test_emb = smodel_test_emb.to(torch.float16)


Mod(smodel_test_emb)
smodel_test_emb.util.map(
    name_module = ["query", 'value'] + ["key", 'dense'])


# activate in the last layers
submod_test = smodel_test_emb.model.encoder.layer[-config["la"]:]
Mod(submod_test)
submod_test.util.activate(
    layer_names= ["query", 'value'],
    key = 'set',
    config = {
        'name':'test',
        "r":4,
        "alpha":4*(0.1)
        })

model = smodel_test_emb
submod_test
smodel_test_emb = smodel_test_emb.to(torch.float16)



res_just_one = []

for lo, wo in zip([lod1, lod2, lod3, lod4, lod6],
                  [w_1, w_2, w_3, w_4, w_6]):
  load_adapt_head(model, submod_test, *wo)
  _=model.cuda()
  res = eval_pipeline(model,*lo,
                      device="cuda", save=None)
  pprint(res)
  _=model.cpu()
  res_just_one.append(res)




###########################

smodel_test_emb = SentenceEmbedding(
    copy.deepcopy(model_ref_t),
    with_head = False,
    normalize=False,
    )



Mod(smodel_test_emb)
smodel_test_emb.util.map(
    name_module = ["query", 'value'] + ["key", 'dense'])


# activate in the last layers
submod_test = smodel_test_emb.model.encoder.layer[-config["la"]:]
Mod(submod_test)
submod_test.util.activate(
    layer_names= ["query", 'value'],
    key = 'set',
    config = {
        'name':'test',
        "r":r,
        "alpha":r*(0.1)
        })



smodel_test_emb = smodel_test_emb.to(torch.float16)
model = smodel_test_emb


load_adapt_head(model, submod_test, *w)


res_merged = []
for lo, wo in zip([lod1, lod2, lod3, lod4, lod6],
                  [w_1, w_2, w_3, w_4, w_6]):
  _=model.cuda()
  res = eval_pipeline(model,*lo,
                      device="cuda", save=None)
  pprint(res)
  _=model.cpu()
  res_merged.append(res)






In [ ]:
names = [base_dataset1,base_dataset2,base_dataset3,base_dataset4,base_dataset6]

names_d = [os.path.basename(x).split(".")[0] for x in names]




In [ ]:
import matplotlib.pyplot as plt

def plot_lines_with_labels(data_lists, label_list):
    fig, ax = plt.subplots(figsize=(12, 8))

    line_styles = ['-', ':', '--']  # Define line styles for each line
    # colors = ['blue', 'green', 'orange', 'purple']  # Define colors for each line
    # colors = plt.cm.viridis(np.linspace(0, 1, 7))
    colors = ['green', 'red','blue', 'purple', 'brown']

    legds=[]
    for j, data_list in enumerate(data_lists):
        color = colors[j % len(colors)]  # Cycle through colors

        auc_values = []  # Store AUC values
        for i, data in enumerate(data_list):
            fpr = data.get("fpr")
            tpr = data.get("tpr")
            roc_auc = data.get("roc_auc")
            line_style = line_styles[i % len(line_styles)]  # Cycle through line styles
            ax.plot(fpr, tpr, lw=2, alpha=0.7,
                    linestyle=line_style, color=color)
            auc_values.append(f'{roc_auc:.2f}')  # Append AUC value to list

        # Format legend entry with all AUC values
        legend_label = f'{label_list[j]} (AUCs: {" / ".join(auc_values)})'
        ls, = ax.plot([], [], lw=2, linestyle='-', color=color, label=legend_label)
        legds.append(ls)

    recall=ax.axhline(y=0.95, color='black', linestyle='-', label=f'Recall 95%', lw=1)

    legend1 = ax.legend(handles=[*legds, recall],
                        loc='lower right', fontsize=14)  # Adjust location and ncols


    ax.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')

    line1, = ax.plot([], [], lw=2, linestyle=line_styles[0], color="black", label="Base")
    line2, = ax.plot([], [], lw=2, linestyle=line_styles[1], color="black", label="Lora")
    line3, = ax.plot([], [], lw=2, linestyle=line_styles[2], color="black", label="Híbrido")

    # Create a legend for the first line.
    legend2 = ax.legend(handles=[line1, line2, line3],
                        loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, fontsize=14)  # Adjust location and ncols



    ax.add_artist(legend1)
    # ax.add_artist(legend2)

    # Add the legend manually to the Axes.

    ax.set_xlabel('Falso Positivo (%)', fontsize=16, weight="bold")
    ax.set_ylabel('Verdadeiro Positivo (%)', fontsize=16, weight="bold")
    plt.yticks(fontsize=18)
    plt.xticks(fontsize=18)


# Test the function
i=0
data_lists = [
    [res_cru[i]["observed"], res_just_one[i]["observed"], res_merged[i]["observed"]],
    [res_cru[i+1]["observed"], res_just_one[i+1]["observed"], res_merged[i+1]["observed"]],
    [res_cru[i+2]["observed"], res_just_one[i+2]["observed"], res_merged[i+2]["observed"]],
    [res_cru[i+3]["observed"], res_just_one[i+3]["observed"], res_merged[i+3]["observed"]],
    [res_cru[i+4]["observed"], res_just_one[i+4]["observed"], res_merged[i+4]["observed"]],
    # Add more data lists as needed
]

label_list = [
    f"{names_d[i]}",
    f"{names_d[i+1]}",
    f"{names_d[i+2]}",
    f"{names_d[i+3]}",
    f"{names_d[i+4]}",
    ]  # Provide labels for each data set

plot_lines_with_labels(data_lists, label_list)
# '-', '--', '-.', ':', 'None', ' ', '', 'solid', 'dashed', 'dashdot', 'dotted'


In [ ]:
dfs = []


for i in range(5):
  df = pd.DataFrame({"awss": [res_cru[i]["95"]["awss"]],
                     "wss": [res_cru[i]["95"]["wss"]],
                    "dataset": [names_d[i]],
                     "método":"base"}
                    )

  dfs.append(df)

for i in range(5):
  df = pd.DataFrame({"awss": [res_just_one[i]["95"]["awss"]],
                     "wss": [res_just_one[i]["95"]["wss"]],
                    "dataset": [names_d[i]],
                     "método":"Lora"}
                    )

  dfs.append(df)
for i in range(5):
  df = pd.DataFrame({"awss": [res_merged[i]["95"]["awss"]],
                     "wss": [res_merged[i]["95"]["wss"]],
                    "dataset": [names_d[i]],
                     "método":"Híbrido"}
                    )

  dfs.append(df)



result_df = pd.concat(dfs, ignore_index=True)
# print(result_df)

result_df=result_df.pivot(index='dataset', columns=['método'], values=['awss'])
print(result_df)
print(result_df.to_latex(float_format="%.2f"))


# # Pivot the DataFrame
# pivot_df = result_df.pivot_table(index='dataset', columns='método', values=['awss', 'wss'])



# print(pivot_df.to_latex())


# # Rearrange columns
# pivot_df = pivot_df.swaplevel(axis=1).sort_index(axis=1)
# pivot_df

In [ ]:


res_cru[i]["95"]["awss"]



res_just_one[i]["95"]
res_merged[i]["95"]


#### 🔱 optimization


##### quantization

In [ ]:
# model_base_t = copy.deepcopy(model_ref_t)


##########################################################################
# ## Quant
smodel_mod_t.util.activate(
        layer_names= ["query", 'value']+["key", 'dense'],
        key = 'set',
        config = {
            'name': 'Quant',
            'bits': 4,
            "copyw":True,
            "base":"base_linear"
            })

smodel_mod_t.util.activate(
        layer_names= ["key", 'dense'] + ["query", 'value'],
        key = 'quantize',
        Modular=Quantize_Linear)

##########################################################################

## lora
submod.util.activate(
        layer_names= ["query", 'value'],
        key = 'set', config= {'name': 'test',
                              'base': 'Quant'}
)


##########################################################################
smodel_mod_t.cuda()

smodel_mod_t.eval()
res = eval_pipeline(smodel_mod_t,
                    train_dataloader=train_dataloader_eval,
                    test_dataloader=test_dataloader_eval,
              device="cuda")

from pprint import pprint
pprint(res)

smodel_mod_t.to("cpu")
torch.cuda.empty_cache()

# export_json(res, "export/diagnostic/Qlora.json")

In [ ]:
# m = copy.deepcopy(smodel_mod_t)

# # del m
# m.util.activate(
#         layer_names= ["key", 'dense'] + ["query", 'value'],
#         key = 'export',
#         Modular=QLinear)


# m.util.activate(
#         layer_names= ["key", 'dense'] + ["query", 'value'],
#         key = 'del_layer',
#         config={
#             "",
#             "base":"Quant"
#         })

# m

# # torch.save(m.state_dict(), "model.bin")

##### sparse

In [ ]:
smodel_mod_t.util.activate(
        layer_names= ["query", 'value', "key"],
        key = 'set',
        config = {
            'name': 'Prune',
            's': 0.7,
            'acum': True,
            "copyw":True
            })


smodel_mod_t.util.activate(
        layer_names= ['dense'],
        key = 'set',
        config = {
            'name': 'Prune',
            's': 0.5,
            'acum': True,
            "copyw":True
            })

smodel_mod_t.to("cuda")
get_batch_labels(
    smodel_mod_t,
    train_dataloader_eval)
smodel_mod_t.to("cpu")


smodel_mod_t.util.activate(
        layer_names= ['dense']+["query", 'value', "key"],
        key = 'prune',
        Modular=Prune
        )

smodel_mod_t.util.activate(
        layer_names= ["key", 'dense'] + ["query", 'value'],
        key = 'set',
        config = {
            'name': 'Prune',
            'fake_prune': True,
            })

## lora
submod.util.activate(
        layer_names= ["query", 'value'],
        key = 'set', config= {'name': 'test',
                              'base': 'Prune'
                              }
)




smodel_mod_t.cuda()
smodel_mod_t.eval()
res = eval_pipeline(smodel_mod_t,
                    train_dataloader=train_dataloader_eval,
                    test_dataloader=test_dataloader_eval,
              device="cuda")
smodel_mod_t.to("cpu")
torch.cuda.empty_cache()


########
m = copy.deepcopy(smodel_mod_t)

m.util.activate(
        layer_names= ['dense']+["query", 'value', "key"],
        key = 'prune',
        Modular=Prune
        )
m.model.encoder.layer[0].attention.self.query.Mix.Prune.main.weight

sub_copy = m.model.encoder.layer[-3:]

## lora
Mod(sub_copy)
m.util.activate(
        layer_names= ["key", 'dense'] + ["query", 'value'],
        key = 'del_layer',
        config={
            "name":["base_linear", "Quant"],
            "base":"Prune"
        })

sub_copy.util.activate(
        layer_names= ["query", 'value'],
        key = 'set', config= {'name': 'test',
                              'base': 'Prune'}
)

sub_copy.util.activate(
        layer_names= ["query", 'value'],
        key = 'del_layer',
        config={
            "name":["Adapt"],
            "base":"Prune"
        })
##########


res["sparse"]= 100 - count_non_zero_params(m)
del m

from pprint import pprint
pprint(res)




##### 🎒 Distillation

[La mini paper](https://arxiv.org/pdf/2002.10957.pdf) + [Sbert](https://arxiv.org/pdf/1908.10084.pdf)

finetuned in 1B pairs


###### pipe

In [ ]:
tk = lambda x: len(tokenizer_t.encode(x))

tk_len = df_test.text.map(tk)

tk_len.hist()
df_rest_unlb = df_test[tk_len > 100]

In [ ]:
df_train_unlabel = df_rest_unlb.sample(64)


In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm



model_kind = "SIMModel"


total_batch = 32
accumulation_steps = 1

batch_size=int(total_batch/accumulation_steps)


######
n_epocs= 2
n_pairs = 30
n_unlabaled = 8

lr = 3e-3


###
lora_alpha=1
r=8
tau = 0.1

###
la = 3
rest = 12 - la

######





# pd.concat([df_test.sample(n_unlabaled),df_train])
# just labeled data
n_epocs= 2
sim_data_1 = SIMDataset(df_train, tokenize_batch_t, n_pairs)
train_dataloader_1 = DataLoader(sim_data_1, shuffle=True, batch_size=batch_size)

interation_1 = int(len(sim_data_1)/batch_size)
total_iterations_1 = n_epocs*(interation_1)


# just unlabeled data

n_epocs= 2
sim_data_2 = SIMDataset(df_train_unlabel, tokenize_batch_t, n_pairs = 300, n_pairs_random = True)
train_dataloader_2 = DataLoader(sim_data_2, shuffle=True, batch_size=batch_size)

interation_2 = int(len(sim_data_2)/batch_size)
total_iterations_2 = n_epocs*(interation_2)

# unlabeled + labeld data
accumulation_steps= 1
batch_size=int(64/accumulation_steps)
n_epocs=2
sim_data_3 = SIMDataset(pd.concat([df_train_unlabel, df_train]),
                        tokenize_batch_t,
                        n_pairs = 600,
                        n_pairs_random = True)
train_dataloader_3 = DataLoader(sim_data_3, shuffle=True, batch_size=batch_size)

interation_3 = int(len(sim_data_3)/batch_size)
total_iterations_3 = n_epocs*(interation_3)



In [ ]:
Mod = ModularUtils()
model_name = 'sentence-transformers/paraphrase-MiniLM-L3-v2'
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'
modelref_s = AutoModel.from_pretrained(model_name)
tokenizer_s = AutoTokenizer.from_pretrained(model_name)

Mod.freeze_layers(modelref_s, freeze=True)
Mod.freeze_layers(smodel_mod_t, freeze=True)

model_base_s = copy.deepcopy(modelref_s)
model_mod_s = Mod.copy_arch(model_base_s)
smodel_mod_s = SentenceEmbedding(
    model_mod_s,
    hidden =384,
    with_head=True
    )

# proj = nn.Sequential(
#       nn.Linear(384, 384),
#       # nn.GELU(),
#       # nn.Linear(384, 384)
#     )

# smodel_mod_s.head.add_module(
#     'proj',
#     proj
# )

# model_base_t
# model_mod_t
# smodel_mod_t


max_seq_length = 512

tokenize_s = lambda train_text: tokenizer_s(train_text,
    padding="max_length",
    max_length=max_seq_length,
    truncation=True,
    return_tensors="pt")


tokenize_batch_s = lambda train_text: tokenizer_s.batch_encode_plus(
    train_text,
    add_special_tokens=True,
    padding='max_length',
    max_length=max_seq_length,
    truncation=True,
    return_tensors='pt',
    return_attention_mask=True
)

###### model


Mod = ModularUtils()

# base model and the lora adapter model
Mod.freeze_layers(model_ref_t, freeze=True)


model_base_s = copy.deepcopy(model_ref_s)
model_mod_s = Mod.copy_arch(model_base_s)


Mod(smodel_mod_s)


# apply ModMix in all corresponding values

smodel_mod_s.util.map(
    Modular=ModMix,
    name_module = ["query", 'value'])



# Active Lora just in some layers

# get the last layers and apply the lora
submod = smodel_mod_s.get_submodule('encoder.layer')[-la:]


# Mod(submod)
submod.util.activate(
    layer_names= ["query", 'value'],
    key = 'set',
    name= 'test', config = {
    'alpha': 1,
    'r':32})





print_trainable_params(smodel_mod_s)

# l2 = loss_distill_to_light(
#     model_s = smodel_mod_s,
#     model_t = smodel_mod_t

# )



# with just df_train
trainer_1 = Trainer(losses=few_shot_loss_distill(
    model_s = smodel_mod_s,
    model_t = smodel_mod_t,
    loss=1).to('cuda'),
                  optimizer=Adam(smodel_mod_s.parameters(), lr=lr),
                  accumulation_steps = accumulation_steps,
                  train_dataloader=train_dataloader_1,
                  total_iterations=total_iterations_1,
                  device = 'cuda')

# with just df_unlabel
# trainer_2 = Trainer(losses=few_shot_loss_distill(
#     model_s = smodel_mod_s,
#     model_t = smodel_mod_t,
#     loss=1).to('cuda'),
#                   optimizer=Adam(smodel_mod_s.parameters(), lr=lr),
#                   accumulation_steps = accumulation_steps,
#                   train_dataloader=train_dataloader_2,
#                   total_iterations=total_iterations_2,
#                   device = 'cuda')

# with  dftrain and df_unlabel
# 5e-4
trainer_3 = Trainer(losses=few_shot_loss_distill(
    model_s = smodel_mod_s,
    model_t = smodel_mod_t,
    loss=0).to('cuda'),
                  optimizer=Adam(smodel_mod_s.parameters(), lr=lr),
                  accumulation_steps = accumulation_steps,
                  train_dataloader=train_dataloader_3,
                  total_iterations=total_iterations_3,
                  device = 'cuda')




#### train

print(f"Quantidade de amostras (rotuladas)\n {len(sim_data_1)}")
print(f"Quantidade de exemplos (rotuladas)\n {len(df_train)}")
print(f"Quantidade de amostras (nao rotuladas-par)\n {len(sim_data_2)}")
print(f"Quantidade de exemplos (nao rotuladas)\n {len(df_train_unlabel)}")
print(f"Quantidade de amostras (não + rotuladas-par)\n {len(sim_data_3)}")
# print(f"Quantidade de exemplos (nao rotuladas)\n {len(df_train_unlabel)}")



print(submod[0].attention.self.value.loraA.weight)

# model_s2.to("cpu")
# torch.cuda.empty_cache()
# trainer_2.train()
# trainer_1.train()

# trainer_3.train()

# smodel_mod_s.to('cuda')
# res = eval_pipeline(smodel_mod_s,
#               (df_train,pd.concat([df_train,exs_test])),
#               is_SLRC=(model_kind == "SLRC"),
#               batch_size=300,
#                     tokenize = tokenize_batch_s)

# from pprint import pprint
# pprint(res)

# smodel_mod_s.to("cpu")
# torch.cuda.empty_cache()


smodel_mod_s.to('cuda')
trainer_1.train()
print(submod[0].attention.self.value.loraA.weight)

res = eval_pipeline(smodel_mod_s,
              (df_train,pd.concat([df_train,exs_test])),
              is_SLRC=(model_kind == "SLRC"),
              batch_size=300,
                    tokenize = tokenize_batch_s)

from pprint import pprint
pprint(res)

smodel_mod_s.to("cpu")
torch.cuda.empty_cache()


In [ ]:
for n, mo in list(submod.named_children()):
  mo.attention.self.query.merge()
  mo.attention.self.value.merge()


proj2 = nn.Sequential(
      nn.Linear(384, 384),
      # nn.GELU(),
      # nn.Linear(384, 384)
    )

Mod.freeze_layers(smodel_mod_s, freeze=True)
smodel_mod_s.model = model_base_s


submod2,_ =  Mod.app_Modular(
        submod, # Modular
        ["base_linear"], # linear layes (type_layers)
        )



# del smodel_mod_s.head.proj

smodel_mod_s.head.add_module(
    'proj',
    proj2
)


l3 = loss_sim(smodel_mod_s,
              tau = 0.05)
print_trainable_params(l3)

print(submod[0].attention.self.value.loraA.weight)

trainer_4 = Trainer(losses=l3.to('cuda'),
                  optimizer=Adam(l3.parameters(), lr=3e-5),
                  accumulation_steps = accumulation_steps,
                  train_dataloader=train_dataloader_1,
                  total_iterations=total_iterations_1,
                  device = 'cuda')
trainer_4.train()


print(submod[0].attention.self.value.loraA.weight)


smodel_mod_s.to('cuda')




res = eval_pipeline(smodel_mod_s,
              (df_train,pd.concat([df_train,exs_test])),
              is_SLRC=(model_kind == "SLRC"),
              batch_size=300,
                    tokenize = tokenize_batch_s)

from pprint import pprint
pprint(res)

smodel_mod_s.to("cpu")
torch.cuda.empty_cache()

https://github.com/intel/neural-compressor?tab=readme-ov-file



## Inferece

In [ ]:
# !pip install optimum[onnxruntime-gpu]

In [ ]:
# import onnxruntime as rt

# #define the priority order for the execution providers
# # prefer CUDA Execution Provider over CPU Execution Provider
# EP_list = ['CUDAExecutionProvider', 'CPUExecutionProvider']

# # initialize the model.onnx
# sess = rt.InferenceSession("model.onnx", providers=EP_list)

# # get the outputs metadata as a list of :class:`onnxruntime.NodeArg`
# output_name = sess.get_outputs()[0].name

# # get the inputs metadata as a list of :class:`onnxruntime.NodeArg`
# input_name = sess.get_inputs()[0].name

# # inference run using image_data as the input to the model
# detections = sess.run([output_name], {input_name: image_data})[0]

# print("Output shape:", detections.shape)

# 🌐 Main results

- [x] escolher o modelo dentro de 1 dataset
- [x] usar modelo dentro de todos os datasets/ avaliar metodos de otimização e metricas
- [x] salvar loras e avaliar performance geral se fosse apenas 1 lora

- [ ] avaliar aprendizado continuo em apenas um dataset
- [ ] ?avaliar modelo destilado

- [ ] falar sobre estimativa dos arquivos, quantidade de uso de memoria e velocidades e possibilidades de uso e deploy como modelo geral de transformaçãp

## dataset statistics

In [ ]:
# import os

# def list_files_in_folders(base_path):
#     folder_dict = {}

#     def list_files(directory):
#         return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

#     def list_directories(directory):
#         return [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]

#     for folder in list_directories(base_path):
#         folder_path = os.path.join(base_path, folder)
#         files_in_folder = list_files(folder_path)
#         folder_dict[folder] = files_in_folder

#     return folder_dict

# # Substitua '/content/content/slr_data_tcc' pelo seu caminho real
# result_dict = list_files_in_folders('/content/content/slr_data_tcc')

# # Imprima o resultado
# for folder, files in result_dict.items():
#     print(f"{folder}: {files}")


In [ ]:
import glob
import os

In [ ]:
import glob
import os
from pprint import pprint

# base_path = 'data/content/slr_data_tcc'
base_path = './content/slr_data_tcc'

# Paths and names of the domains datasets
# path = 'SLR_data'
paths_cohen = glob.glob(f"{base_path}/cohen/*.csv", recursive=True)
pprint(paths_cohen)

paths_swift = glob.glob(f"{base_path}/SWIFT systematic review data/*.csv", recursive=True)
pprint(paths_swift)


# names = [os.path.basename(p) for p in paths]




In [ ]:
names_r = {
    "Opiods": "Opioids",
    "ACEInhibitors": "ACE Inhibitors",
    "AtypicalAntipsychotics": "Atypical Antipsychotics",
    "BetaBlockers": "Beta Blockers",
    "UrinaryIncontinence": "Urinary Incontinence",
    "OralHypoglycemics": "Oral Hypoglycemics",
    "ProtonPumpInhibitors": "Proton Pump Inhibitors",
    "CalciumChannelBlockers": "Calcium Channel Blockers",
    "SkeletalMuscleRelaxants": "Skeletal Muscle Relaxants",
    "SkeletalMuscleRelaxants": "Skeletal Muscle Relaxants"
    }

In [ ]:
import os
import matplotlib.pyplot as plt

tk = lambda x: len(tokenizer_t_ref.encode(x))

names = []
tokens_len_0 = []
tokens_len_1 = []

for p in paths_cohen:
    n = os.path.basename(p).split(".")[0]
    if n in list(names_r.keys()):
      names.append(names_r[n])
    else:
      names.append(n)
    df = preprocess_csv_file(p)
    tokens_len_0.append(df[df["label"] == 0].text.map(tk))
    tokens_len_1.append(df[df["label"] == 1].text.map(tk))

for p in paths_swift:
    n = os.path.basename(p).split(".")[0]
    if n in list(names_r.keys()):
      names.append(names_r[n])
    else:
      names.append(n)
    df = preprocess_csv_file(p)
    tokens_len_0.append(df[df["label"] == 0].text.map(tk))
    tokens_len_1.append(df[df["label"] == 1].text.map(tk))


In [ ]:
import matplotlib.patches as mpatches

plt.figure(figsize=(10, 6))

box_width = 0.4

box_0 = plt.boxplot(tokens_len_0, positions=[pos - box_width / 2 for pos in range(len(names))],
                    widths=box_width, patch_artist=True, boxprops=dict(facecolor='lightcoral'))
box_1 = plt.boxplot(tokens_len_1, positions=[pos + box_width / 2 for pos in range(len(names))],
                    widths=box_width, patch_artist=True, boxprops=dict(facecolor='skyblue'))

# Customizing colors for the median line
for median in [box_0['medians'], box_1['medians']]:
    for median_line in median:
        median_line.set(color='black', linewidth=2)

plt.xticks([pos for pos in range(len(names))], [f'{name}' for name in names],
           rotation=45, ha='right', fontsize=14, weight='bold')

# Adjusting horizontal padding of tick labels
plt.xticks(ha='right', rotation_mode='anchor', fontsize=14, weight='bold')
plt.yticks(fontsize=12)

plt.ylabel('Quant. de Tokens', fontsize=14, weight='bold')
# plt.xlabel('Banco de Dados', fontsize=14, weight='bold')

# Creating legend patches
legend_patches = [mpatches.Patch(facecolor='skyblue', label='Label 1'),
                  mpatches.Patch(facecolor='lightcoral', label='Label -1')]
plt.legend(handles=legend_patches, loc='upper left', fontsize='large')

plt.grid(True)

plt.tight_layout()

# Save the plot as PDF
plt.savefig('boxplot_tokens.pdf')

plt.show()


In [ ]:
# Given list
data_list = [('ACE Inhibitors', (2214, 167, 2047)),
 ('ADHD', (781, 80, 701)),
 ('Antihistamines', (277, 87, 190)),
 ('Atypical Antipsychotics', (999, 329, 670)),
 ('Beta Blockers', (1819, 266, 1553)),
 ('Calcium Channel Blockers', (1069, 246, 823)),
 ('Estrogens', (337, 77, 260)),
 ('NSAIDs', (348, 81, 267)),
 ('Opioids', (1717, 41, 1676)),
 ('Oral Hypoglycemics', (462, 134, 328)),
 ('Proton Pump Inhibitors', (1171, 220, 951)),
 ('Skeletal Muscle Relaxants', (1318, 26, 1292)),
 ('Statins', (2659, 150, 2509)),
 ('Triptans', (573, 200, 373)),
 ('Urinary Incontinence', (271, 65, 206)),
 ('Drug Reviews (COHEN et al., 2006)', (16015, 2169, 13846)),
 ('Bisphenol A (BPA)', (7093, 102, 6991)),
 ('Fluoride and Neurotoxicity', (3870, 49, 3821)),
 ('Neurophatic pain', (29202, 5009, 24193)),
 ('PFOA/PFOS', (5950, 95, 5855)),
 ('Transgenerational', (46147, 606, 45541)),
 ('SWIFT (??)', (92262, 5861, 86401))]

# Compute proportion of positives for each entry
proportions = []
for entry in data_list:
    total, positive, _ = entry[1]
    proportion = positive / total
    proportions.append(proportion)

# Display the proportions
for entry, proportion in zip(data_list, proportions):
    print(f"{entry[0]}: &{proportion:.4f}")


## avaliation all

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from torch.optim.lr_scheduler import LambdaLR
#######
import pandas as pd
import os
from glob import glob
from pprint import pprint

def process_files(base_path):
    paths = glob(f"{base_path}/**/*.csv", recursive=True)

    # Criar listas para armazenar os DataFrames e os caminhos dos arquivos
    dfs = []
    file_paths = []

    # Iterar sobre os caminhos dos arquivos
    for file_path in paths:
        try:
            # Carregar o DataFrame
            df = pd.read_csv(file_path)  # ou pd.read_excel(file_path) para arquivos Excel

            # Renomear colunas se necessário
            if 'label_included' in df.columns:
                df.rename(columns={'label_included': 'label'}, inplace=True)
            if 'labels' in df.columns:
                df.rename(columns={'labels': 'label'}, inplace=True)

            # Adicionar o caminho e nome do arquivo como colunas extras
            df['file_path'] = file_path
            df['file_name'] = os.path.basename(file_path)

            # Adicionar o DataFrame à lista
            dfs.append(df)
            file_paths.append(file_path)

        except Exception as e:
            print(f"Erro ao processar o arquivo {file_path}: {str(e)}")

    # Concatenar todos os DataFrames em um único DataFrame
    final_df = pd.concat(dfs, ignore_index=True)

    # Imprimir informações sobre o DataFrame final
    print("Informações sobre o DataFrame final:")
    print(final_df.info())

    # Exemplo de visualização das primeiras linhas do DataFrame final
    print("\nPrimeiras linhas do DataFrame final:")
    print(final_df.head())

    # Preencher valores nulos nas colunas 'title' e 'abstract'
    for col in ["title", "abstract"]:
        final_df.loc[final_df[col].isna(), col] = ""

    # Criar a coluna 'text' concatenando 'title' e 'abstract'
    final_df['text'] = final_df['title'] + "; " + final_df['abstract']

    return final_df


def model_train_validation(data_path, model_name, trys=1, config={}, just_notrain=False):
    for i_try in range(trys):
      ###### Init mdeol
        ##########################################################################

        model_ref_t = AutoModel.from_pretrained(model_name)
        tokenizer_t_ref = AutoTokenizer.from_pretrained(model_name)


        max_seq_length = 512

        tokenize_batch_t = lambda train_text: tokenizer_t_ref.batch_encode_plus(
            train_text,
            add_special_tokens=True,
            padding='max_length',
            max_length=max_seq_length,
            truncation=True,
            return_tensors='pt',
            return_attention_mask=True
        )

        data_name = os.path.basename(data_path).split(".")[0]

        base = "/content/gdrive/MyDrive/Projetos/TCC"
        base_save = f"{base}/export/diagnostic/{model_name}/{data_name}"
        create_repo_if_not_exist(base_save+"/")
        file_path = data_path


        # model_ref_t, model_name


      ###### Init data
        ##########################################################################


        df = preprocess_csv_file(file_path)

        tk = lambda x: len(tokenizer_t_ref.encode(x))
        tk_len = df.text.map(tk)

        df_rest = df[tk_len > 20]

        df_train = df_rest.groupby('label').sample(8)
        df_test = df[~df.index.isin(df_train.index)]

        exs_test = pd.concat([df_train, df_test])


        sim_data = SIMDataset(df_train, tokenize_batch_t, config["n_pairs"],
                              # emb= emb, device="cuda", pttn="* d a", b=300
                              )
        train_dataloader = DataLoader(sim_data,shuffle=True,
                                      batch_size=config["batch_size"])
        test_dataloader_eval = DataLoader(shuffle=False, batch_size=300,
                    dataset=SIMDataset(
                        exs_test, tokenize_batch_t, 0,
                        # emb= emb, device="cuda", pttn="* d a"
                              )
                    )

        train_dataloader_eval = DataLoader(shuffle=False, batch_size=len(df_train),
                    dataset=SIMDataset(
                        df_train, tokenize_batch_t, 0,
                        # emb= emb, device="cuda", pttn="* d a"
                              )
                    )

      ####
        ##########################################################################
        ########################    Nothing   ####################################
        ##########################################################################

        if just_notrain:

          save = f"{base_save}/No_train/{i_try}/"
          create_repo_if_not_exist(save)

          smodel_mod_t = SentenceEmbedding(
              copy.deepcopy(model_ref_t),
              with_head = False)

          smodel_mod_t.to("cuda")
          res = eval_pipeline(smodel_mod_t,
                              train_dataloader=train_dataloader_eval,
                              test_dataloader=test_dataloader_eval,
                              device="cuda",
                              save= save)

          export_json(res, f"{save}Metrics_{i_try}.json")

          from pprint import pprint

          pprint(res)


          smodel_mod_t.to("cpu")
          torch.cuda.empty_cache()

        else:

        ##### modular
          ##########################################################################
          ########################    LoRA   #######################################
          ##########################################################################

          Mod = ModularUtils(Modular=ModMix)
          Mod.freeze(module=model_ref_t)

          model_base_t = copy.deepcopy(model_ref_t)
          model_mod_t = Mod.copy_arch(model_base_t)

          # model_mod_t.encoder.layer = model_mod_t.encoder.layer[-config["la"]:]
          # submod = model_mod_t.encoder.layer
          submod = model_mod_t.encoder.layer[-config["la"]:]

          ###### model
          smodel_mod_t = SentenceEmbedding(
              model_mod_t,
              with_head = True)


          # Mod(smodel_mod_t)
          Mod(smodel_mod_t)
          smodel_mod_t.util.map(
              name_module = ["query", 'value'] + ["key", 'dense'])


          Mod(submod)
          # activate in the last layers
          submod.util.activate(
              layer_names= ["query", 'value'],
              key = 'set',
              config = {
                  'name':'test',
                  **config
                  })


          smodel_mod_t.util.trainable_params()


          l1 = loss_sim(smodel_mod_t,tau = config["tau"], device="cuda").to("cuda")



          # ##########################################################################
          # ######## data   #################
          # model_base_t2 = copy.deepcopy(model_ref_t)
          # model_base_t2.encoder.layer = model_base_t2.encoder.layer[:config["rest"]]

          # model_base_t2.to("cuda")
          # model_base_t2.eval()
          # emb = lambda **x: model_base_t2(**x)[0]



          interation = int(len(sim_data)/config["batch_size"])
          total_iterations = config["n_epocs"]*(interation)
          ##########################################################################


          def lr_lambda(epoch):
              return 1 - (epoch / total_iterations)**2


          # Create the linear learning rate scheduler
          optimizer = Adam(smodel_mod_t.parameters(), lr=config["lr"])
          scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)

          #### train
          trainer = Trainer(losses=l1.to('cuda'),
                            optimizer=optimizer,
                            scheduler=scheduler,
                            accumulation_steps = config["accumulation_steps"],
                            train_dataloader=train_dataloader,
                            total_iterations=total_iterations,
                            device = 'cuda')



          print(f"Quantidade de amostras\n {len(sim_data)}")
          print(f"Quantidade de exemplos\n {len(df_train)}")

          w = submod[0].attention.self.value.Mix['Adapt'].loraA

          print(submod)
          print()
          # print(w.weight)

          trainer.train()

          # print(w.weight)
          ##########################################################################

          # model_base_t2.to("cpu")



          save = f"{base_save}/Lora/{i_try}/"
          create_repo_if_not_exist(save)


          res = eval_pipeline(smodel_mod_t,
                              train_dataloader=train_dataloader_eval,
                              test_dataloader=test_dataloader_eval,
                              device="cuda",
                              save= save)

          export_json(res, f"{save}Metrics_{i_try}.json")

          from pprint import pprint

          pprint(res)


          smodel_mod_t.to("cpu")
          torch.cuda.empty_cache()

        ##### modular
          ##########################################################################
          ########################    QLoRA   ######################################
          ##########################################################################
          # model_base_t = copy.deepcopy(model_ref_t)


          ##########################################################################
          # ## Quant
          smodel_mod_t.util.activate(
                  layer_names= ["query", 'value']+["key", 'dense'],
                  key = 'set',
                  config = {
                      'name': 'Quant',
                      'bits': 4,
                      "copyw":True,
                      "base":"base_linear"
                      })

          smodel_mod_t.util.activate(
                  layer_names= ["key", 'dense'] + ["query", 'value'],
                  key = 'quantize',
                  Modular=Quantize_Linear)

          ##########################################################################

          ## lora
          submod.util.activate(
                  layer_names= ["query", 'value'],
                  key = 'set', config= {'name': 'test',
                                        'base': 'Quant'}
          )


          ##########################################################################

          save = f"{base_save}/Quant/{i_try}/"
          create_repo_if_not_exist(save)

          smodel_mod_t.cuda()
          smodel_mod_t.eval()
          res = eval_pipeline(smodel_mod_t,
                              train_dataloader=train_dataloader_eval,
                              test_dataloader=test_dataloader_eval,
                        device="cuda",
                              save=save)

          from pprint import pprint
          pprint(res)
          export_json(res, f"{save}Metrics_{i_try}.json")

          smodel_mod_t.to("cpu")
          torch.cuda.empty_cache()



        ##### modular
          ##########################################################################
          ########################    SPARSE   #####################################
          ##########################################################################

          smodel_mod_t.util.activate(
                  layer_names= ["query", 'value', "key"],
                  key = 'set',
                  config = {
                      'name': 'Prune',
                      's': 0.6,
                      'acum': True,
                      "copyw":True
                      })


          smodel_mod_t.util.activate(
                  layer_names= ['dense'],
                  key = 'set',
                  config = {
                      'name': 'Prune',
                      's': 0.6,
                      'acum': True,
                      "copyw":True
                      })

          smodel_mod_t.to("cuda")
          get_batch_labels(
              smodel_mod_t,
              train_dataloader_eval)
          smodel_mod_t.to("cpu")


          smodel_mod_t.util.activate(
                  layer_names= ['dense']+["query", 'value', "key"],
                  key = 'prune',
                  Modular=Prune
                  )

          smodel_mod_t.util.activate(
                  layer_names= ["key", 'dense'] + ["query", 'value'],
                  key = 'set',
                  config = {
                      'name': 'Prune',
                      'fake_prune': True,
                      })

          ## lora
          submod.util.activate(
                  layer_names= ["query", 'value'],
                  key = 'set', config= {'name': 'test',
                                        'base': 'Prune'
                                        }
          )




          smodel_mod_t.cuda()
          smodel_mod_t.eval()


          save = f"{base_save}/Prune/{i_try}/"
          create_repo_if_not_exist(save)


          res = eval_pipeline(smodel_mod_t,
                              train_dataloader=train_dataloader_eval,
                              test_dataloader=test_dataloader_eval,
                        device="cuda",
                              save=save)
          smodel_mod_t.to("cpu")
          torch.cuda.empty_cache()


          ########
          m = copy.deepcopy(smodel_mod_t)

          m.util.activate(
                  layer_names= ['dense']+["query", 'value', "key"],
                  key = 'prune',
                  Modular=Prune
                  )
          m.model.encoder.layer[0].attention.self.query.Mix.Prune.main.weight

          sub_copy = m.model.encoder.layer[-3:]

          ## lora
          Mod(sub_copy)
          m.util.activate(
                  layer_names= ["key", 'dense'] + ["query", 'value'],
                  key = 'del_layer',
                  config={
                      "name":["base_linear", "Quant"],
                      "base":"Prune"
                  })

          sub_copy.util.activate(
                  layer_names= ["query", 'value'],
                  key = 'set', config= {'name': 'test',
                                        'base': 'Prune'}
          )

          sub_copy.util.activate(
                  layer_names= ["query", 'value'],
                  key = 'del_layer',
                  config={
                      "name":["Adapt"],
                      "base":"Prune"
                  })
          ##########


          res["sparse"] = 100 - count_non_zero_params(m)
          export_json(res, f"{save}Metrics_{i_try}.json")

          del m

          pprint(res)


        ##### modular
          ##########################################################################
          ##########################################################################
          ##########################################################################

          submod.util.activate(
              layer_names= ["query", 'value'],
              key = 'export',
              Modular=Lora)

          save = f"{base_save}/Lora/{i_try}/"
          export_model(submod.util.export_mod(Modular=Lora),
                      f"{save}/lora_w.bin")


In [ ]:
import glob
import os
from pprint import pprint
base_path = '/content/content/slr_data_tcc'
# Paths and names of the domains datasets
# path = 'SLR_data'
paths1 = glob.glob(f"{base_path}/SWIFT systematic review data/*.csv", recursive=True)
paths2 = glob.glob(f"{base_path}/cohen/*.csv", recursive=True)
paths=[*paths1, *paths2]


# names = [os.path.basename(p).split(".")[0] for p in paths]

In [ ]:
config = {
    "total_batch": 16,
    "accumulation_steps": 1,
    "batch_size":int(16/1),
###
    "n_epocs": 2,
    "n_pairs" : 40,
    "lr" : 7e-3,
###
    "alpha":4*(0.1),
    "r":4,
    "tau" : 0.2,
###
    "la" : 3,
    "rest" : 12 - 3,
    "samples":8
           }

seed = 43
np.random.seed(seed)

# Set seed for PyTorch
torch.manual_seed(seed)

# Set seed for CUDA if available
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)


# base_dataset = "cohen/Opiods.csv"
model_name = 'allenai/specter2_base'
    # model_name = 'sentence-transformers/paraphrase-MiniLM-L3-v2'
    # model_name = 'sentence-transformers/all-MiniLM-L12-v2'
    # model_name = 'allenai/scibert_scivocab_uncased'

# for p in paths[1:-1]:
for p in paths:
  model_train_validation(p, model_name, config=config, just_notrain = False)

### *metrics*

In [ ]:
import glob
import os
from pprint import pprint
base_path = '/content/gdrive/MyDrive/Projetos/TCC/export/diagnostic/allenai/specter2_base'
# Paths and names of the domains datasets
# path = 'SLR_data'

paths_notrain = glob.glob(f"{base_path}/**/No_train/0/*.json", recursive=True)
paths_lora = glob.glob(f"{base_path}/**/Lora/0/*.json", recursive=True)
paths_quant = glob.glob(f"{base_path}/**/Quant/0/*.json", recursive=True)
paths_prune = glob.glob(f"{base_path}/**/Prune/0/*.json", recursive=True)
# pprint(paths)





# names = [os.path.basename(p).split(".")[0] for p in paths]

In [ ]:
names_r = {
    "Opiods": "Opioids",
    "ACEInhibitors": "ACE Inhibitors",
    "AtypicalAntipsychotics": "Atypical Antipsychotics",
    "BetaBlockers": "Beta Blockers",
    "UrinaryIncontinence": "Urinary Incontinence",
    "OralHypoglycemics": "Oral Hypoglycemics",
    "ProtonPumpInhibitors": "Proton Pump Inhibitors",
    "CalciumChannelBlockers": "Calcium Channel Blockers",
    "SkeletalMuscleRelaxants": "Skeletal Muscle Relaxants",
    "SkeletalMuscleRelaxants": "Skeletal Muscle Relaxants"
    }

In [ ]:
import matplotlib.pyplot as plt


def json_data(paths):
  metrics = []
  for p in paths:
    da = read_json(p)
    for t in ["95", "observed"]:
      metric = create_subset(da[t], ["wss", "awss", "f1", "R", "prox."])
      metric["method"] = p.split("/")[-3]
      metric["dataset"] = p.split("/")[-4]
      metric["type"] = t
      metrics.append(metric)

  return metrics

data_no_train = pd.DataFrame(json_data(paths_notrain))
data_lora = pd.DataFrame(json_data(paths_lora))
data_quant = pd.DataFrame(json_data(paths_quant))
data_prune = pd.DataFrame(json_data(paths_prune))

data_no_train["dataset"] = data_no_train["dataset"].replace(names_r)
data_lora["dataset"] = data_lora["dataset"].replace(names_r)
data_quant["dataset"] = data_quant["dataset"].replace(names_r)
data_prune["dataset"] = data_prune["dataset"].replace(names_r)



sub_no_train = data_no_train[data_no_train["type"] == "95"]
sub_lora = data_lora[data_lora["type"] == "95"]
sub_quant = data_quant[data_quant["type"] == "95"]
sub_prune = data_prune[data_prune["type"] == "95"]



plt.figure(figsize=(5, 8))

plt.grid(True)
plt.scatter( sub_lora['awss'],sub_lora['dataset'], color='blue',label="Lora", alpha=0.9,marker="3", s=350)
plt.scatter( sub_quant['awss'],sub_quant['dataset'], color='green', label="Quant. 4bit", alpha=0.9,  marker="3", s=350)
plt.scatter( sub_prune['awss'],sub_prune['dataset'], color='red', label="Prune", alpha=0.9, marker="3", s=350)
plt.scatter( sub_no_train['awss'],sub_no_train['dataset'], color='black',label="Base", alpha=1, marker='*', s=60)
# plt.ylabel('Dataset', fontsize=14)
plt.xlabel('AWSS', fontsize=14, weight="bold")
plt.xticks(fontsize=12)  # Set font size for x-axis ticks
plt.yticks(fontsize=14,  weight="bold")  # Set font size for y-axis ticks
plt.legend(fontsize=11)
# plt.title('Scatter Plot of AWSS by Dataset')
plt.savefig('all.pdf')
plt.show()




# print(lora_data.round(3)[["awss", "dataset", "type", "R"]].pivot_table(index='dataset', columns='type').style.to_latex())




## svd

In [ ]:
import torch
import torch.nn as nn

# Define the input size and output size for the linear layers
input_size = 2
output_size = 4

# Define the first linear layer
linear_layer1 = nn.Linear(input_size, output_size)

# Define the second linear layer
linear_layer2 = nn.Linear(input_size, output_size)

delta= linear_layer1.weight - linear_layer2.weight

# Concatenate the weights of the two linear layers

w1 = linear_layer1.weight
w2 = linear_layer2.weight

wm = torch.cat((w1, w2), dim=1)

linear_layer3 = nn.Linear(2 * input_size, output_size, bias=False)

with torch.no_grad():
    linear_layer3.weight.copy_(wm)


linear_layer3.weight, w1, w2


In [ ]:
a =0.5


m1 = w1.norm(p=2, dim=0, keepdim=True)
m2 = w2.norm(p=2, dim=0, keepdim=True)

delta = a*w1/m1 + (1-a)*w2/m2
delta
# m = delta.norm(p=2, dim=0, keepdim=True)

# delta_v = delta / m

In [ ]:
r=2
U, S, V = torch.svd(delta)

At = U[:, :r]
Bt = torch.matmul(torch.diag(S[:r]), V[:r, :])

w = torch.matmul(At, Bt)

w*(a*m1+ (1-a)*m2)



In [ ]:

# modular
Mod = ModularUtils(Modular=ModMix)

# base model and the lora adapter model
Mod.freeze(module=model_ref_t)


model_base_t = copy.deepcopy(model_ref_t)
model_base_t = SentenceEmbedding(
    model_ref_t,
    with_head = True
    )

model_mod_t = Mod.copy_arch(smodel_mod_t)








# save = f"{base_save}/NoTrain/"
# create_repo_if_not_exist(save)


# smodel_mod_t.eval().to("cuda")
# res = eval_pipeline(smodel_mod_t,
#                     train_dataloader = DataLoader(
#                       SIMDataset(df_train, tokenize_batch_t),
#                       shuffle=False, batch_size=len(df_train)),
#                     test_dataloader = DataLoader(
#                       SIMDataset(exs_test, tokenize_batch_t),
#                       shuffle=False, batch_size=300),
#               device="cuda", perplexity=30,
#               save= save)


# from pprint import pprint
# pprint(res)

# export_json(res, f"{save}Metrics.json")

# smodel_mod_t.to("cpu")

# del smodel_mod_t, model_base_t
# torch.cuda.empty_cache()

## Main pipeline

2.4 . Estimativa de recall
Aqui descrevemos um método estatístico simples para estimar a recordação dada uma lista classificada de referências. Assumimos que um avaliador começa a triagem no topo da lista, fornecendo sequencialmente um rótulo para cada item: “Incluído” ou “Excluído”. Estamos interessados ​​em estimar quando o avaliador atingiu um determinado nível de recordação (digamos, 95%) para que ele/ela possa interromper a triagem e perceber os benefícios da classificação de documentos (ao mesmo tempo que mantém a confiança de que a maioria dos documentos relevantes foram descobertos). ).

O método funciona examinando a extensão de intervalos consecutivos de documentos excluídos que ocorrem entre cada documento relevante durante a triagem. Os comprimentos desses vãos fornecem uma base para estimar a probabilidade local de relevância do documento. Se os documentos fossem selecionados em ordem aleatória, esses comprimentos de intervalo seguiriam uma distribuição geométrica com uma taxa (“probabilidade de sucesso”) refletindo a frequência subjacente de documentos relevantes (incluídos) na lista de referência. Da mesma forma, se considerarmos os intervalos entre um conjunto de mais de dois documentos consecutivos incluídos, então o comprimento total do intervalo, uma soma de variáveis ​​aleatórias geométricas independentes, seguiria a distribuição binomial negativa. Portanto, os comprimentos observados dos documentos excluídos podem ser usados ​​para estimar o parâmetro de taxa subjacente para os documentos incluídos. Esta taxa pode então, por sua vez, ser utilizada para estimar o número de documentos relevantes restantes na lista de documentos não rastreados. Na prática, à medida que o avaliador avança pela lista classificada de referências, as lacunas entre os documentos relevantes tenderão a aumentar em extensão, porque, por definição, as listas classificadas são carregadas antecipadamente com documentos relevantes. Por outras palavras, embora a distribuição binomial negativa presuma que os “sucessos” são distribuídos de forma independente e aleatória, neste cenário, a taxa de sucesso deve ser decrescente (em vez de fixa ou crescente). Como resultado, este método tende a resultar em uma estimativa conservadora de recordação. Ou seja, a recordação verdadeira obtida é muitas vezes ligeiramente superior ao seu valor estimado.

Este algoritmo possui um parâmetro de “ajuste” que chamamos de “lookback”, denotado por δ . Este parâmetro determina o número de vãos considerados na estimativa da taxa de inclusão. Totalizamos a distância do vão, D , (em número de documentos) entre a posição de triagem atual e o δ -ésimo documento incluído anteriormente. Se a taxa de inclusão para documentos restantes for p , e supondo (hipoteticamente) que os documentos foram amostrados aleatoriamente para triagem, então

Com essas informações, o recall estimado é calculado com base em
do seguinte modo:
onde
é uma estimativa de p com base no D observado , TP é o número de documentos relevantes identificados até o momento pelo rastreador e U é o número de documentos restantes não rastreados.

3.2.2 . Efeito do comprimento da lista
Na Tabela A.4 , mostramos o custo resultante de diversas simulações utilizando a distribuição de pontuação BPA. Para cada simulação, a taxa geral de inclusão foi definida em p  = 0,15 e o lookback, δ  = 2. Usando esses parâmetros, simulamos listas classificadas de vários comprimentos. Os custos resultantes são calculados em média em 5 tentativas. Conforme mostrado na tabela, listas mais longas estão associadas a custos menores. Isto porque, para uma determinada taxa de inclusão local, existe um número fixo de documentos que devem ser analisados ​​para se obter uma boa estimativa dessa taxa. À medida que o comprimento da lista aumenta, este número fixo torna-se menor como uma fração da carga total de triagem.

3.2.3 . Efeito da taxa de inclusão
A Tabela A.5 mostra o efeito da taxa de inclusão global, p , no custo. Para cada simulação, o comprimento da lista foi fixado em 10.000 documentos e o lookback, δ  = 2. Os custos resultantes são calculados em média em 5 tentativas. Em geral, quanto menor for a taxa global de inclusão, maior será o custo. Para estimar uma pequena taxa de inclusão é necessário triar mais documentos, o que por sua vez aumenta o custo.

3.2.4 . Efeito do lookback δ
A Tabela A.6 mostra o efeito da variação sistemática do lookback, δ , nos dados simulados a partir do conjunto de dados de dor neuropática. Para esses experimentos, o comprimento da lista foi fixado em 30.000 e a taxa de inclusão, p , foi fixada em 0,17. Os custos resultantes foram calculados em média em 5 tentativas. A tabela mostra que, em geral, o custo é uma função crescente de δ . Observe que para δ  = 1 e δ  = 2, o custo foi realmente negativo, indicando que a recordação verdadeira obtida está na verdade abaixo dos 95% desejados. Conseqüentemente, escolher valores de δ muito pequenos pode levar a subestimar a recuperação.

A Figura A.6 ilustra a relação entre a variabilidade da estimativa de recall (e o custo correspondente) e o parâmetro δ . No Painel (a) δ  = 1 e no Painel (b) δ  = 100. Observe a variabilidade drasticamente diminuída na estimativa de recall para o valor mais alto de δ . Em geral, aumentar δ diminui a variabilidade da estimativa de recordação. Portanto, há um trade-off entre a variabilidade do custo e o valor esperado do custo. Ignorando a variabilidade, a configuração ideal para δ é, idealmente, o menor valor tal que o custo esperado seja não negativo. Em geral, δ maior pode ser ideal quando p for maior. Por outro lado, quando p é muito baixo ou a lista é curta, pode não ser aconselhável usar um δ grande .

**Negative-binomial distribuition**

$X\sim NBin(\delta,p)$

- $\delta$ - Número de sucessos na amostra
- $p$ - Probabilidade de sucesso
- $x_i$ - Quantas amostras para ter o ultimo sucesso

$$\begin{aligned}
p(x={x_i})&= {{\delta+{x_i}-1}\choose{{x_i}}}(1-p)^{x_i} p^\delta \\
l(p;x)&\propto {x_i}\log(1-p)+ r \log (p) \\
l(p;x_1,...,x_n) &\propto \sum{x_i}\log(1-p)+ n\delta \log (p) \\
\end{aligned}$$

Portanto

$$\begin{aligned}
{\partial \over \partial p}l(p;x_1,...,x_n) &=  {-\sum{x_i} \over (1-p)} + {n\delta \over p}=0\\
\implies \hat{p} =  {\delta \over \delta+ \bar{x}}
\end{aligned}$$



Recall estimado com $\hat{p}\times U = FN$

$$R={TP\over TP +\hat{p}\times U}$$
- U Número de documentos restantes não rastreados
- TP Número de documentos Relevantes

$$AWSS = {TN\%} -(1-FP\%)$$


## tests

In [ ]:
def get_index_of_ith_one(i, random_list, maximum=None):
    """
    Generate a random list of 0s and 1s and return the index of the i-th occurrence of '1'.

    Parameters:
    i : int
        The occurrence of '1' to find.
    random_list : list
        The list of 0s and 1s.

    Returns:
    int
        The index of the i-th occurrence of '1'.
        Returns -1 if i exceeds the number of '1's in the list.
    """
    ones_count = 0
    zeros_count = 0
    last_one_index = -1
    if maximum is None:
      maximum = len(random_list)


    for index, value in enumerate(random_list):
        if index > maximum:
          break

        if value == 1:
            ones_count += 1
            if ones_count == i:
                last_one_index = index
                break
        else:
            zeros_count += 1




    if last_one_index != -1:
        return last_one_index, zeros_count, ones_count
    else:
        return -1, zeros_count, ones_count


d = i_th_occurrence = 10
U=length = 10000

# U*0.01
random_list = np.random.binomial(1,0.01,length)


last_one, D, ones_count = get_index_of_ith_one(i_th_occurrence, random_list, 100)


print(f"The index of the {i_th_occurrence}-th occurrence of '1' is: {last_one+1}")
from pprint import pprint


e=0.0001
p = (ones_count+e)/(ones_count+e+D)

d=ones_count
TP = 10
FN = p*(U-d)
r = (TP+d)/(TP+d+FN)

print(f"FN: {round(FN)}")
print(f"phat: {round(p,2)}")
print(f"D: {D}")
print(f"Ones: {ones_count}")
print(f"recall: {r}")

TP = 10
FN = 1

TP/(TP + FN)

# print()




In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import recall_score

# Gerando dados de exemplo
np.random.seed(42)
X = np.random.rand(1000, 1) * 10  # 1000 amostras entre 0 e 10
y = (X[:, 0] > 5).astype(int)  # Rotulando como 1 se X > 5, senão 0

# Inicializando modelo de regressão logística
modelo = LogisticRegression(max_iter=1000)

# Número inicial de amostras no conjunto de treinamento
n = 2

indices_amostras_treino = np.random.choice(range(1000), n, replace=False)

X_treino = X[indices_amostras_treino]
y_treino = y[indices_amostras_treino]

delta = 2

# p_hat = delta / (delta+ mean(x) )
d = 1 # um sucesso
x=200 # no 200 rank


TP = 100
U= 1000




p = d/(d+x)


R = TP / (TP + p*U)

print(p)
print(R)





for tentativa in range(5):

    # Treinando o modelo
    modelo.fit(X_treino, y_treino)

    # ramain data
    id_test = np.isin(range(len(X)), indices_amostras_treino)
    X_test, y_test = X[~id_test] , y[~id_test]

    # Obtendo probabilidades para todas as amostras
    probabilidades = modelo.predict_proba(X_test)[:, 1]

    preds = modelo.predict(X_test)

    # Ordenando amostras com base nas probabilidades
    indices_ordenados = np.argsort(probabilidades)[::-1]
    next_sample = indices_ordenados[:n]
    indices_amostras_treino = np.concatenate([indices_amostras_treino, next_sample])



    U = len(y_test) # remain to label
    TP = np.sum(y_treino)

    x= get_index_of_ith_one(delta, y_test[indices_ordenados])

    p = d/(
        +x)
    R = TP /(TP + p*U)

    print(f"phat: {p}")
    print(f"over R: {R}")

    X_treino, y_treino = X[indices_amostras_treino], y[indices_amostras_treino]


    print('X_treino:',len(X_treino))
    acuracia = modelo.score(X_test, y_test)
    recall = recall_score(y_test, preds)
    print(f"Tentativa {tentativa + 1}, Acurácia: {acuracia:.2f}")
    print(f"Recall: {recall:.2f}")

    print(f"####"*100)





# Após o loop, você pode usar o modelo treinado conforme necessário.


## main

In [ ]:
directory_path = 'data'

# List files in the directory
with os.scandir(directory_path) as entries:
    file_paths = [entry.path for entry in entries if entry.is_file()]

t = LazyLoadDataset(file_paths)

DataLoader(t, batch_size=1, shuffle=False)

t[0]

In [ ]:
from torch.utils.data import DataLoader

def get_index_of_ith_one(i, random_list, maximum=None):
    """
    Generate a random list of 0s and 1s and return the index of the i-th occurrence of '1'.

    Parameters:
    i : int
        The occurrence of '1' to find.
    random_list : list
        The list of 0s and 1s.

    Returns:
    int
        The index of the i-th occurrence of '1'.
        Returns -1 if i exceeds the number of '1's in the list.
    """
    ones_count = 0
    zeros_count = 0
    last_one_index = -1
    if maximum is None:
      maximum = len(random_list)


    for index, value in enumerate(random_list):
        if index > maximum:
          break

        if value == 1:
            ones_count += 1
            if ones_count == i:
                last_one_index = index
                break
        else:
            zeros_count += 1




    if last_one_index != -1:
        return last_one_index, zeros_count, ones_count
    else:
        return -1, zeros_count, ones_count


def apply_modular_operations(model_ref_t, la, lora_alpha, r):
    """Apply ModularUtils operations."""
    # Initialize ModularUtils
    Mod = ModularUtils(Modular=ModMix)

    # Freeze base model and the lora adapter model
    Mod.freeze(module=model_ref_t)

    # Create a copy of the base model
    model_base_t = copy.deepcopy(model_ref_t)

    # Copy architecture using ModularUtils
    model_mod_t = Mod.copy_arch(model_base_t)

    # Create SentenceEmbedding with head
    smodel_mod_t = SentenceEmbedding(model_mod_t, with_head=True)

    # Get weights of the head layer
    head_weights = smodel_mod_t.head[1].weight

    # Apply ModMix in all corresponding values
    Mod(smodel_mod_t)
    smodel_mod_t.util.map(name_module=["query", 'value'] + ["key", 'dense'])

    # Active Lora just in some layers
    # Get the last layers and apply the lora
    submod = smodel_mod_t.get_submodule('encoder.layer')[-la:]
    Mod(submod)

    # Activate Lora in the last layers
    submod.util.activate(
        layer_names=["query", 'value'],
        key='set',
        config={
            'alpha': lora_alpha,
            'r': r,
            'name': 'test'
        }
    )

    return smodel_mod_t


def prepare_training(X_train, tokenize_func, n_pairs, batch_size, n_epochs, smodel, tau, lr, accumulation_steps):
    temp = SIMDataset(X_train, tokenize_func, n_pairs)
    train_dataloader = DataLoader(temp,
                                  shuffle=True,
                                  batch_size=batch_size)

    interation = int(len(temp)/batch_size)
    total_iterations = n_epocs*(interation)



    trainer = Trainer(losses=loss_sim(smodel, tau = tau).to('cuda'),
                      optimizer=Adam(smodel.parameters(), lr=lr),
                      accumulation_steps = accumulation_steps,
                      train_dataloader=train_dataloader,
                      total_iterations=total_iterations,
                      device = 'cuda').train()



import torch
import numpy as np
from sklearn.linear_model import LogisticRegression

def train_logistic_regression(X_train, Y_train, X_test, n_pairs, smodel):
    emd_dataloader = DataLoader(SIMDataset(X_train, tokenize_batch_t), batch_size=batch_size)

    all_emd = []
    for x in emd_dataloader:
        all_emd.append(smodel(x))
    emd_x = torch.cat(all_emd)

    head_model = LogisticRegression(max_iter=1000)
    head_model.fit(emd_x, Y_train)

    probabilities = head_model.predict_proba(X_test)[:, 1]
    preds = head_model.predict(X_test)

    indices_ordered = np.argsort(probabilities)[::-1]
    idx_train = np.concatenate([idx_train, indices_ordered[:n_pairs]])

    X_train_new, Y_train_new = X_train[idx_train], Y_train[idx_train]

    return X_train_new, Y_train_new, preds, probabilities

import copy



    # return trainer

# Usage:
# trainer = prepare_training(X_train, tokenize_batch_t, n_pairs, batch_size, n_epochs, smodel, tau, lr, accumulation_steps)


smodel_test = copy.deepcopy(smodel)
smodel_test=smodel_test.to(torch.bfloat16)
submod = smodel_test.encoder.layer[-config["la"]:]

_ = main_pipe_train(smodel_test, submod, *lod_train1 ,config)
# w_1 = export_adapt_head(smodel_test, submod)

##### model

n_pairs = 40
n_epocs=2
batch_size=16
accumulation_steps=1
tau=0.2
lr=0.
########################

### gen seed




base_dataset5 = "cohen/Opiods.csv"
file_path = f"data/content/slr_data_tcc/{base_dataset5}"
df = preprocess_csv_file(file_path)



X = df
Y = df['label']
idx_all = X.index

seed = X.groupby('label').sample(4)

df_train = seed
idx_train = df_train.index
df_test = df[~df.index.isin(idx_train)]

X_train, Y_train = (df_train, df_train['label'])


for tentativa in range(5):

############ start here
X_test = X[~idx_all.isin(idx_train)]
Y_test = Y[~idx_all.isin(idx_train)]


idx_test = df_test.index


#####################
# first train sbert

lod1 = (train_dataloader_eval1, test_dataloader_eval1)
lod_train1= (df_train1, train_dataloader, *lod1)



temp = SIMDataset(X_train, tokenize_batch_t, n_pairs)
train_dataloader = DataLoader(temp,
                              shuffle=True,
                              batch_size=batch_size)

interation = int(len(temp)/batch_size)
total_iterations = n_epocs*(interation)


trainer = Trainer(losses=loss_sim(model, tau = tau).to('cuda'),
                  optimizer=Adam(model.parameters(), lr=lr),
                  accumulation_steps = accumulation_steps,
                  train_dataloader=train_dataloader,
                  total_iterations=total_iterations,
                  device = 'cuda').train()

#####################

# second train sbert

emd_dataloader = DataLoader(SIMDataset(X_train, tokenize_batch_t),
                              batch_size)



all = []
for x in emd_dataloader:
  all.append(model(x))

emd_x =  torch.concat(all)

head_model = LogisticRegression(max_iter=1000)
head_model.fit(emd_x, Y_train)


probabilidades = head_model.predict_proba(X_test)[:, 1]
preds = modelo.predict(X_test)

indices_ordenados = np.argsort(probabilidades)[::-1]
idx_train = np.concatenate([idx_train, indices_ordenados[:n]])


X_train, y_train = X[idx_train], Y[idx_train]

#####################
# metrics

U = len(Y_test) # remain to label
TP = np.sum(Y_train)

x= get_index_of_ith_one(delta, Y_test[indices_ordenados])

p = d/(d+x)
R = TP /(TP + p*U)


metrics(probabilidades, Y_test, recall_target=0.95)

acuracia = modelo.score(X_test, Y_test)
recall = recall_score(Y_test, preds)

print(f"Tentativa {tentativa + 1}, Acurácia: {acuracia:.2f}")
print(f"Recall: {recall:.2f}")


# Teste

In [ ]:
!pip install einops einop

In [ ]:
import einops
import torch
import numpy as np
from einop import einop


y = torch.randn(size=(2,2,2,3))

abs(y)

In [ ]:
einop(abs(y),"b (i i0) j k->(b i) i0 (j k)", i0=2)

In [ ]:
einop(einop(abs(y),"b i j k->(b i) (j k)"),
      "(i i0) j -> i i0 j", i0=2)

In [ ]:
abs(y) / einop(abs(y),"b m n d->b m n 1", 'sum')

In [ ]:
einop(y,"b m n d-> b (m n) d")

# abs(y) / einop(abs(y),"b m n d->1 m n d", 'sum')

In [ ]:
abs(y) / einop(abs(y),"b m n d->1 m n d", 'sum')

In [ ]:
einop(y,"b m n d->b m d", 'min')


In [ ]:
!pip install -q einops

In [ ]:
from einops import pack, unpack
import numpy as np
# h, w = 2, 2
# image_rgb is 3-dimensional (h, w, 3) and depth is 2-dimensional (h, w)
a = torch.Tensor(np.random.random([32,2]))
b = torch.Tensor(np.random.random([32,2]))
# but we can stack them


In [ ]:
c, ps = pack([a, b], '* b')

print(c.shape)
# c

In [ ]:
d = unpack(c, [[1]]*7,"* a b")

print(d[0].shape)
d

In [ ]:
torch.einsum('i j k, i j k-> j k', d[0:2]).shape

In [ ]:
from einops import rearrange, reduce, repeat

x = torch.randn((2,4,4))
x

In [ ]:
reduce(x, 'b (h h2) (w w2) -> b h w', 'max', h2=2, w2=1).shape
reduce(x, 'b (h h2) (w w2) -> b h w', 'max', h2=2, w2=1)

In [ ]:
from einops.layers.torch import EinMix as Mix

# other stuff we use
import torch
from torch import nn

x = torch.randn((2,4,4))

In [ ]:


print("## x:\n", x)
print("#####"*3)

a = Mix('b i j->b i i0', weight_shape='j i0',
        # j =4,
        # i =1,
        i0 =2,
        j =4,
        # j0=2,
        # k= 2,
        )

w = list(a.parameters())

w[0].requires_grad=False

w[0][:] = torch.Tensor([1])

print(f"### \n(x) {x.shape}\n(w) {w[0].shape} \n(y) {a(x).shape}")
print("#####"*3)
print("## w:\n", w)
print("#####"*3)
print("## aw(x):\n", a(x))
x
a

In [ ]:
# w[0].requires_grad=False


# w[0][:,[0,3]] = torch.Tensor([0])

 (x[0][:1,:] * w[0]).sum()

In [ ]:
torch.sum(x[0][:2,0:2]*w[0][0])
torch.sum(x[0][2:,0:2]*w[0][0])
# torch.sum(x[0][0,:]*w[0][0]) + torch.sum(x[0][1,:]*w[0][1])
# x[0]*w[0][0,:,:]

In [ ]:


print("## x:\n", x)
print("#####"*3)

a = Mix('b i (j j0)-> b i', weight_shape='i j0',
        i =2,
        # j=2,
        # i0=4,
        j0=2,
        )


w = list(a.parameters())
print(f"### \n(x) {x.shape}\n(w) {w[0].shape} \n(y) {a(x).shape}")
print("#####"*3)
print("## w:\n", w)
print("#####"*3)
print("## aw(x):\n", a(x))


In [ ]:
import torch
from torch import nn

# x = torch.randn((2,4,4))

x = torch.Tensor([
    [1,0],
    [0,1],
    ])

a = torch.eye(2) * torch.Tensor([1,1])


x.shape, a.shape

torch.matmul(x, a)



In [ ]:
x = torch.randn((2,2))


a,b = torch.linalg.eig(x)
a,b

In [ ]:
x

In [ ]:
b @ a